In [ ]:
import scipy.stats as stats
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import tikzplotlib

In [ ]:
table = pd.DataFrame()

for point in np.arange(0, 3.10, step = .01):
    
    single_dec = math.trunc(point * 10) / 10
    double = math.trunc(point * 100 + .0001) / 100  # +.0001 for float margin of error
    assert double >= single_dec
    hundreds = double - single_dec
    cdf_val = stats.norm.cdf(double)
    
    is_even = (10*single_dec % 2) == 0
    
    helper = ''
    if is_even:
        helper = r'\cellcolor{gray!20}'
    
    table.loc[str(single_dec), str('{:+.2f}'.format(hundreds))] = helper + '{:.4f}'.format(cdf_val)
    
assert table.isnull().sum().sum() == 0

In [ ]:
#print(table.to_latex(escape = False))
s = table.style.to_latex(hrules = True)

In [ ]:
wd = os.getcwd().rstrip("/python")

%cd ~

In [ ]:
# Add illustration

x = np.linspace(-3,3, 1000)
fig, ax = plt.subplots()
y = stats.norm.pdf(x)
ax.plot(x, y)
ax.set_aspect(6)

for spine in 'left', 'right', 'top':
    ax.spines[spine].set_visible(False)
ax.set_yticks([])
ylim = ax.get_ylim()
xlim = ax.get_xlim()

z_score = 1
x_vals = x[x<z_score]
y_vals = y[0:len(x_vals)]
ax.fill_between(x_vals, y_vals, alpha = 0.15)

x = np.linspace(-4,4, 2000)
y = stats.norm.pdf(x)
ax.plot(x, y, color = 'C0')

ax.set_ylim(0, ylim[1])
ax.set_xlim(xlim)

# annotate area

ax.text(1, stats.norm.pdf(1)/1.2,
        s = '{:.4f}  '.format(stats.norm.cdf(1)),
        ha = 'right',
        va = 'center')

ax.set_xticks([1])
ax.set_xticklabels(['$z=1$'])

ax.plot([1,1], [0, stats.norm.pdf(1)], color = 'black', ls = '-', lw = 0.4)
#ax.plot([1], [stats.norm.pdf(1)], color = 'black', marker = '.')

#tikz_plot_beta = tikzplotlib.get_tikz_code()
#tikzplotlib.save('testplot.tex')

#print(s)
#plt.savefig('ztable_plot.pdf')
plt.show()

In [ ]:
tikz_plot = r"""
% This file was created with tikzplotlib v0.10.1.
% axis modifications from ChatGPT
\begin{tikzpicture}

\definecolor{darkgray176}{RGB}{176,176,176}
\definecolor{steelblue31119180}{RGB}{31,119,180}

\begin{axis}[
tick align=outside,
tick pos=left,
x grid style={darkgray176},
xmin=-3, xmax=3,
xtick style={color=black},
xtick={1},
xticklabels={\(\displaystyle z=1\)},
ymin=0, ymax=0.418665913227224,
ytick=\empty,
axis y line= none,
axis x line = bottom,
axis line style = {-}
%axis lines = bottom
]
\path [fill=steelblue31119180, fill opacity=0.15]
(axis cs:-3,0)
--(axis cs:-3,0.00443184841193801)
--(axis cs:-2.99399399399399,0.0045123438886223)
--(axis cs:-2.98798798798799,0.00459413567812293)
--(axis cs:-2.98198198198198,0.00467724132352912)
--(axis cs:-2.97597597597598,0.00476167854213413)
--(axis cs:-2.96996996996997,0.00484746522600703)
--(axis cs:-2.96396396396396,0.00493461944254309)
--(axis cs:-2.95795795795796,0.0050231594349922)
--(axis cs:-2.95195195195195,0.00511310362296472)
--(axis cs:-2.94594594594595,0.00520447060291445)
--(axis cs:-2.93993993993994,0.0052972791485979)
--(axis cs:-2.93393393393393,0.0053915482115097)
--(axis cs:-2.92792792792793,0.00548729692129329)
--(axis cs:-2.92192192192192,0.00558454458612664)
--(axis cs:-2.91591591591592,0.00568331069308223)
--(axis cs:-2.90990990990991,0.00578361490846092)
--(axis cs:-2.9039039039039,0.00588547707809912)
--(axis cs:-2.8978978978979,0.00598891722764874)
--(axis cs:-2.89189189189189,0.00609395556282928)
--(axis cs:-2.88588588588589,0.00620061246965171)
--(axis cs:-2.87987987987988,0.00630890851461339)
--(axis cs:-2.87387387387387,0.00641886444486359)
--(axis cs:-2.86786786786787,0.00653050118833907)
--(axis cs:-2.86186186186186,0.0066438398538691)
--(axis cs:-2.85585585585586,0.00675890173124943)
--(axis cs:-2.84984984984985,0.0068757082912846)
--(axis cs:-2.84384384384384,0.00699428118579803)
--(axis cs:-2.83783783783784,0.00711464224760944)
--(axis cs:-2.83183183183183,0.00723681349047888)
--(axis cs:-2.82582582582583,0.00736081710901693)
--(axis cs:-2.81981981981982,0.00748667547856039)
--(axis cs:-2.81381381381381,0.00761441115501307)
--(axis cs:-2.80780780780781,0.00774404687465088)
--(axis cs:-2.8018018018018,0.00787560555389093)
--(axis cs:-2.7957957957958,0.0080091102890238)
--(axis cs:-2.78978978978979,0.00814458435590864)
--(axis cs:-2.78378378378378,0.00828205120963033)
--(axis cs:-2.77777777777778,0.00842153448411835)
--(axis cs:-2.77177177177177,0.00856305799172663)
--(axis cs:-2.76576576576577,0.00870664572277382)
--(axis cs:-2.75975975975976,0.00885232184504357)
--(axis cs:-2.75375375375375,0.00900011070324404)
--(axis cs:-2.74774774774775,0.00915003681842624)
--(axis cs:-2.74174174174174,0.00930212488736056)
--(axis cs:-2.73573573573574,0.00945639978187097)
--(axis cs:-2.72972972972973,0.00961288654812628)
--(axis cs:-2.72372372372372,0.00977161040588793)
--(axis cs:-2.71771771771772,0.00993259674771377)
--(axis cs:-2.71171171171171,0.0100958711381172)
--(axis cs:-2.70570570570571,0.0102614593126814)
--(axis cs:-2.6996996996997,0.0104293871771275)
--(axis cs:-2.69369369369369,0.0105996808063366)
--(axis cs:-2.68768768768769,0.0107723664433257)
--(axis cs:-2.68168168168168,0.0109474704981749)
--(axis cs:-2.67567567567568,0.0111250195469083)
--(axis cs:-2.66966966966967,0.0113050403303254)
--(axis cs:-2.66366366366366,0.011487559752784)
--(axis cs:-2.65765765765766,0.0116726048809339)
--(axis cs:-2.65165165165165,0.0118602029423997)
--(axis cs:-2.64564564564565,0.0120503813244144)
--(axis cs:-2.63963963963964,0.0122431675724008)
--(axis cs:-2.63363363363363,0.0124385893885022)
--(axis cs:-2.62762762762763,0.0126366746300606)
--(axis cs:-2.62162162162162,0.0128374513080428)
--(axis cs:-2.61561561561562,0.0130409475854126)
--(axis cs:-2.60960960960961,0.0132471917754508)
--(axis cs:-2.6036036036036,0.0134562123400202)
--(axis cs:-2.5975975975976,0.013668037887776)
--(axis cs:-2.59159159159159,0.0138826971723226)
--(axis cs:-2.58558558558559,0.0141002190903132)
--(axis cs:-2.57957957957958,0.0143206326794947)
--(axis cs:-2.57357357357357,0.0145439671166962)
--(axis cs:-2.56756756756757,0.0147702517157603)
--(axis cs:-2.56156156156156,0.0149995159254171)
--(axis cs:-2.55555555555556,0.0152317893271017)
--(axis cs:-2.54954954954955,0.0154671016327119)
--(axis cs:-2.54354354354354,0.015705482682309)
--(axis cs:-2.53753753753754,0.015946962441759)
--(axis cs:-2.53153153153153,0.0161915710003145)
--(axis cs:-2.52552552552553,0.016439338568137)
--(axis cs:-2.51951951951952,0.0166902954737598)
--(axis cs:-2.51351351351351,0.0169444721614894)
--(axis cs:-2.50750750750751,0.0172018991887471)
--(axis cs:-2.5015015015015,0.0174626072233491)
--(axis cs:-2.4954954954955,0.017726627040725)
--(axis cs:-2.48948948948949,0.0179939895210751)
--(axis cs:-2.48348348348348,0.0182647256464652)
--(axis cs:-2.47747747747748,0.0185388664978589)
--(axis cs:-2.47147147147147,0.0188164432520877)
--(axis cs:-2.46546546546547,0.0190974871787576)
--(axis cs:-2.45945945945946,0.0193820296370931)
--(axis cs:-2.45345345345345,0.0196701020727171)
--(axis cs:-2.44744744744745,0.0199617360143675)
--(axis cs:-2.44144144144144,0.0202569630705496)
--(axis cs:-2.43543543543544,0.0205558149261244)
--(axis cs:-2.42942942942943,0.0208583233388324)
--(axis cs:-2.42342342342342,0.0211645201357533)
--(axis cs:-2.41741741741742,0.0214744372097002)
--(axis cs:-2.41141141141141,0.021788106515549)
--(axis cs:-2.40540540540541,0.0221055600665036)
--(axis cs:-2.3993993993994,0.0224268299302945)
--(axis cs:-2.39339339339339,0.0227519482253133)
--(axis cs:-2.38738738738739,0.023080947116681)
--(axis cs:-2.38138138138138,0.0234138588122505)
--(axis cs:-2.37537537537538,0.0237507155585446)
--(axis cs:-2.36936936936937,0.0240915496366271)
--(axis cs:-2.36336336336336,0.0244363933579092)
--(axis cs:-2.35735735735736,0.0247852790598887)
--(axis cs:-2.35135135135135,0.025138239101825)
--(axis cs:-2.34534534534535,0.0254953058603477)
--(axis cs:-2.33933933933934,0.0258565117249987)
--(axis cs:-2.33333333333333,0.0262218890937095)
--(axis cs:-2.32732732732733,0.0265914703682128)
--(axis cs:-2.32132132132132,0.0269652879493876)
--(axis cs:-2.31531531531532,0.0273433742325399)
--(axis cs:-2.30930930930931,0.0277257616026172)
--(axis cs:-2.3033033033033,0.0281124824293578)
--(axis cs:-2.2972972972973,0.0285035690623752)
--(axis cs:-2.29129129129129,0.0288990538261777)
--(axis cs:-2.28528528528529,0.0292989690151221)
--(axis cs:-2.27927927927928,0.0297033468883044)
--(axis cs:-2.27327327327327,0.0301122196643845)
--(axis cs:-2.26726726726727,0.0305256195163479)
--(axis cs:-2.26126126126126,0.030943578566203)
--(axis cs:-2.25525525525526,0.0313661288796147)
--(axis cs:-2.24924924924925,0.0317933024604746)
--(axis cs:-2.24324324324324,0.0322251312454088)
--(axis cs:-2.23723723723724,0.0326616470982222)
--(axis cs:-2.23123123123123,0.0331028818042809)
--(axis cs:-2.22522522522523,0.0335488670648327)
--(axis cs:-2.21921921921922,0.0339996344912656)
--(axis cs:-2.21321321321321,0.0344552155993059)
--(axis cs:-2.20720720720721,0.0349156418031548)
--(axis cs:-2.2012012012012,0.0353809444095654)
--(axis cs:-2.1951951951952,0.0358511546118594)
--(axis cs:-2.18918918918919,0.0363263034838852)
--(axis cs:-2.18318318318318,0.0368064219739166)
--(axis cs:-2.17717717717718,0.0372915408984941)
--(axis cs:-2.17117117117117,0.0377816909362074)
--(axis cs:-2.16516516516517,0.0382769026214225)
--(axis cs:-2.15915915915916,0.0387772063379509)
--(axis cs:-2.15315315315315,0.0392826323126641)
--(axis cs:-2.14714714714715,0.0397932106090522)
--(axis cs:-2.14114114114114,0.0403089711207292)
--(axis cs:-2.13513513513514,0.040829943564883)
--(axis cs:-2.12912912912913,0.041356157475674)
--(axis cs:-2.12312312312312,0.0418876421975804)
--(axis cs:-2.11711711711712,0.0424244268786927)
--(axis cs:-2.11111111111111,0.0429665404639572)
--(axis cs:-2.10510510510511,0.0435140116883696)
--(axis cs:-2.0990990990991,0.04406686907012)
--(axis cs:-2.09309309309309,0.0446251409036892)
--(axis cs:-2.08708708708709,0.0451888552528987)
--(axis cs:-2.08108108108108,0.0457580399439133)
--(axis cs:-2.07507507507508,0.0463327225581989)
--(axis cs:-2.06906906906907,0.0469129304254359)
--(axis cs:-2.06306306306306,0.0474986906163891)
--(axis cs:-2.05705705705706,0.0480900299357348)
--(axis cs:-2.05105105105105,0.0486869749148476)
--(axis cs:-2.04504504504505,0.0492895518045451)
--(axis cs:-2.03903903903904,0.0498977865677951)
--(axis cs:-2.03303303303303,0.0505117048723834)
--(axis cs:-2.02702702702703,0.0511313320835454)
--(axis cs:-2.02102102102102,0.0517566932565611)
--(axis cs:-2.01501501501502,0.0523878131293159)
--(axis cs:-2.00900900900901,0.0530247161148276)
--(axis cs:-2.003003003003,0.0536674262937412)
--(axis cs:-1.996996996997,0.0543159674067926)
--(axis cs:-1.99099099099099,0.0549703628472423)
--(axis cs:-1.98498498498498,0.0556306356532813)
--(axis cs:-1.97897897897898,0.0562968085004086)
--(axis cs:-1.97297297297297,0.0569689036937837)
--(axis cs:-1.96696696696697,0.0576469431605548)
--(axis cs:-1.96096096096096,0.0583309484421626)
--(axis cs:-1.95495495495495,0.0590209406866238)
--(axis cs:-1.94894894894895,0.0597169406407933)
--(axis cs:-1.94294294294294,0.0604189686426077)
--(axis cs:-1.93693693693694,0.061127044613312)
--(axis cs:-1.93093093093093,0.0618411880496693)
--(axis cs:-1.92492492492492,0.0625614180161572)
--(axis cs:-1.91891891891892,0.0632877531371501)
--(axis cs:-1.91291291291291,0.0640202115890914)
--(axis cs:-1.90690690690691,0.0647588110926551)
--(axis cs:-1.9009009009009,0.0655035689048992)
--(axis cs:-1.89489489489489,0.0662545018114133)
--(axis cs:-1.88888888888889,0.0670116261184602)
--(axis cs:-1.88288288288288,0.0677749576451152)
--(axis cs:-1.87687687687688,0.0685445117154032)
--(axis cs:-1.87087087087087,0.0693203031504359)
--(axis cs:-1.86486486486486,0.070102346260551)
--(axis cs:-1.85885885885886,0.0708906548374551)
--(axis cs:-1.85285285285285,0.0716852421463716)
--(axis cs:-1.84684684684685,0.0724861209181953)
--(axis cs:-1.84084084084084,0.0732933033416567)
--(axis cs:-1.83483483483483,0.0741068010554964)
--(axis cs:-1.82882882882883,0.0749266251406517)
--(axis cs:-1.82282282282282,0.0757527861124582)
--(axis cs:-1.81681681681682,0.0765852939128673)
--(axis cs:-1.81081081081081,0.077424157902681)
--(axis cs:-1.8048048048048,0.0782693868538078)
--(axis cs:-1.7987987987988,0.07912098894154)
--(axis cs:-1.79279279279279,0.0799789717368538)
--(axis cs:-1.78678678678679,0.0808433421987368)
--(axis cs:-1.78078078078078,0.0817141066665415)
--(axis cs:-1.77477477477477,0.0825912708523695)
--(axis cs:-1.76876876876877,0.0834748398334868)
--(axis cs:-1.76276276276276,0.0843648180447727)
--(axis cs:-1.75675675675676,0.0852612092712038)
--(axis cs:-1.75075075075075,0.0861640166403769)
--(axis cs:-1.74474474474474,0.0870732426150701)
--(axis cs:-1.73873873873874,0.0879888889858462)
--(axis cs:-1.73273273273273,0.0889109568636995)
--(axis cs:-1.72672672672673,0.0898394466727489)
--(axis cs:-1.72072072072072,0.0907743581429781)
--(axis cs:-1.71471471471471,0.0917156903030263)
--(axis cs:-1.70870870870871,0.0926634414730305)
--(axis cs:-1.7027027027027,0.0936176092575221)
--(axis cs:-1.6966966966967,0.0945781905383797)
--(axis cs:-1.69069069069069,0.0955451814678399)
--(axis cs:-1.68468468468468,0.0965185774615691)
--(axis cs:-1.67867867867868,0.0974983731917974)
--(axis cs:-1.67267267267267,0.0984845625805171)
--(axis cs:-1.66666666666667,0.0994771387927487)
--(axis cs:-1.66066066066066,0.100476094229875)
--(axis cs:-1.65465465465465,0.101481420523047)
--(axis cs:-1.64864864864865,0.102493108526664)
--(axis cs:-1.64264264264264,0.103511148311924)
--(axis cs:-1.63663663663664,0.10453552916046)
--(axis cs:-1.63063063063063,0.105566239558052)
--(axis cs:-1.62462462462462,0.106603267188415)
--(axis cs:-1.61861861861862,0.107646598927082)
--(axis cs:-1.61261261261261,0.108696220835366)
--(axis cs:-1.60660660660661,0.109752118154414)
--(axis cs:-1.6006006006006,0.110814275299345)
--(axis cs:-1.59459459459459,0.111882675853491)
--(axis cs:-1.58858858858859,0.112957302562724)
--(axis cs:-1.58258258258258,0.114038137329885)
--(axis cs:-1.57657657657658,0.115125161209309)
--(axis cs:-1.57057057057057,0.116218354401455)
--(axis cs:-1.56456456456456,0.117317696247634)
--(axis cs:-1.55855855855856,0.118423165224848)
--(axis cs:-1.55255255255255,0.119534738940736)
--(axis cs:-1.54654654654655,0.120652394128624)
--(axis cs:-1.54054054054054,0.121776106642695)
--(axis cs:-1.53453453453453,0.122905851453265)
--(axis cs:-1.52852852852853,0.124041602642185)
--(axis cs:-1.52252252252252,0.125183333398351)
--(axis cs:-1.51651651651652,0.126331016013341)
--(axis cs:-1.51051051051051,0.12748462187717)
--(axis cs:-1.5045045045045,0.128644121474174)
--(axis cs:-1.4984984984985,0.129809484379015)
--(axis cs:-1.49249249249249,0.13098067925282)
--(axis cs:-1.48648648648649,0.132157673839447)
--(axis cs:-1.48048048048048,0.133340434961882)
--(axis cs:-1.47447447447447,0.134528928518776)
--(axis cs:-1.46846846846847,0.135723119481111)
--(axis cs:-1.46246246246246,0.136922971889012)
--(axis cs:-1.45645645645646,0.138128448848694)
--(axis cs:-1.45045045045045,0.139339512529552)
--(axis cs:-1.44444444444444,0.140556124161395)
--(axis cs:-1.43843843843844,0.141778244031828)
--(axis cs:-1.43243243243243,0.143005831483783)
--(axis cs:-1.42642642642643,0.14423884491319)
--(axis cs:-1.42042042042042,0.145477241766812)
--(axis cs:-1.41441441441441,0.146720978540224)
--(axis cs:-1.40840840840841,0.147970010775955)
--(axis cs:-1.4024024024024,0.149224293061777)
--(axis cs:-1.3963963963964,0.150483779029159)
--(axis cs:-1.39039039039039,0.151748421351882)
--(axis cs:-1.38438438438438,0.153018171744808)
--(axis cs:-1.37837837837838,0.154292980962825)
--(axis cs:-1.37237237237237,0.155572798799941)
--(axis cs:-1.36636636636637,0.15685757408856)
--(axis cs:-1.36036036036036,0.158147254698914)
--(axis cs:-1.35435435435435,0.159441787538676)
--(axis cs:-1.34834834834835,0.160741118552729)
--(axis cs:-1.34234234234234,0.162045192723125)
--(axis cs:-1.33633633633634,0.163353954069202)
--(axis cs:-1.33033033033033,0.164667345647889)
--(axis cs:-1.32432432432432,0.165985309554181)
--(axis cs:-1.31831831831832,0.167307786921791)
--(axis cs:-1.31231231231231,0.168634717923986)
--(axis cs:-1.30630630630631,0.169966041774598)
--(axis cs:-1.3003003003003,0.171301696729227)
--(axis cs:-1.29429429429429,0.172641620086611)
--(axis cs:-1.28828828828829,0.173985748190197)
--(axis cs:-1.28228228228228,0.175334016429883)
--(axis cs:-1.27627627627628,0.176686359243956)
--(axis cs:-1.27027027027027,0.178042710121214)
--(axis cs:-1.26426426426426,0.179403001603276)
--(axis cs:-1.25825825825826,0.180767165287079)
--(axis cs:-1.25225225225225,0.182135131827571)
--(axis cs:-1.24624624624625,0.183506830940589)
--(axis cs:-1.24024024024024,0.184882191405935)
--(axis cs:-1.23423423423423,0.186261141070638)
--(axis cs:-1.22822822822823,0.187643606852412)
--(axis cs:-1.22222222222222,0.189029514743312)
--(axis cs:-1.21621621621622,0.190418789813581)
--(axis cs:-1.21021021021021,0.191811356215688)
--(axis cs:-1.2042042042042,0.193207137188574)
--(axis cs:-1.1981981981982,0.194606055062081)
--(axis cs:-1.19219219219219,0.196008031261586)
--(axis cs:-1.18618618618619,0.197412986312834)
--(axis cs:-1.18018018018018,0.19882083984696)
--(axis cs:-1.17417417417417,0.200231510605714)
--(axis cs:-1.16816816816817,0.20164491644689)
--(axis cs:-1.16216216216216,0.203060974349942)
--(axis cs:-1.15615615615616,0.204479600421811)
--(axis cs:-1.15015015015015,0.205900709902938)
--(axis cs:-1.14414414414414,0.207324217173486)
--(axis cs:-1.13813813813814,0.20875003575976)
--(axis cs:-1.13213213213213,0.210178078340819)
--(axis cs:-1.12612612612613,0.211608256755296)
--(axis cs:-1.12012012012012,0.213040482008409)
--(axis cs:-1.11411411411411,0.214474664279177)
--(axis cs:-1.10810810810811,0.215910712927831)
--(axis cs:-1.1021021021021,0.217348536503421)
--(axis cs:-1.0960960960961,0.218788042751629)
--(axis cs:-1.09009009009009,0.22022913862277)
--(axis cs:-1.08408408408408,0.221671730279998)
--(axis cs:-1.07807807807808,0.2231157231077)
--(axis cs:-1.07207207207207,0.224561021720095)
--(axis cs:-1.06606606606607,0.226007529970025)
--(axis cs:-1.06006006006006,0.227455150957937)
--(axis cs:-1.05405405405405,0.228903787041065)
--(axis cs:-1.04804804804805,0.230353339842804)
--(axis cs:-1.04204204204204,0.231803710262273)
--(axis cs:-1.03603603603604,0.233254798484074)
--(axis cs:-1.03003003003003,0.23470650398824)
--(axis cs:-1.02402402402402,0.236158725560373)
--(axis cs:-1.01801801801802,0.237611361301974)
--(axis cs:-1.01201201201201,0.239064308640953)
--(axis cs:-1.00600600600601,0.240517464342334)
--(axis cs:-1,0.241970724519143)
--(axis cs:-0.993993993993994,0.243423984643478)
--(axis cs:-0.987987987987988,0.244877139557762)
--(axis cs:-0.981981981981982,0.246330083486184)
--(axis cs:-0.975975975975976,0.247782710046314)
--(axis cs:-0.96996996996997,0.249234912260894)
--(axis cs:-0.963963963963964,0.250686582569819)
--(axis cs:-0.957957957957958,0.252137612842278)
--(axis cs:-0.951951951951952,0.253587894389084)
--(axis cs:-0.945945945945946,0.255037317975163)
--(axis cs:-0.93993993993994,0.256485773832225)
--(axis cs:-0.933933933933934,0.257933151671597)
--(axis cs:-0.927927927927928,0.259379340697227)
--(axis cs:-0.921921921921922,0.260824229618852)
--(axis cs:-0.915915915915916,0.262267706665329)
--(axis cs:-0.90990990990991,0.263709659598126)
--(axis cs:-0.903903903903904,0.265149975724979)
--(axis cs:-0.897897897897898,0.266588541913699)
--(axis cs:-0.891891891891892,0.268025244606136)
--(axis cs:-0.885885885885886,0.269459969832298)
--(axis cs:-0.87987987987988,0.270892603224623)
--(axis cs:-0.873873873873874,0.272323030032391)
--(axis cs:-0.867867867867868,0.273751135136292)
--(axis cs:-0.861861861861862,0.275176803063131)
--(axis cs:-0.855855855855856,0.276599918000677)
--(axis cs:-0.84984984984985,0.278020363812654)
--(axis cs:-0.843843843843844,0.279438024053859)
--(axis cs:-0.837837837837838,0.280852781985427)
--(axis cs:-0.831831831831832,0.282264520590213)
--(axis cs:-0.825825825825826,0.283673122588316)
--(axis cs:-0.81981981981982,0.285078470452718)
--(axis cs:-0.813813813813814,0.286480446425052)
--(axis cs:-0.807807807807808,0.287878932531491)
--(axis cs:-0.801801801801802,0.289273810598746)
--(axis cs:-0.795795795795796,0.29066496227019)
--(axis cs:-0.78978978978979,0.292052269022087)
--(axis cs:-0.783783783783784,0.293435612179928)
--(axis cs:-0.777777777777778,0.294814872934877)
--(axis cs:-0.771771771771772,0.29618993236032)
--(axis cs:-0.765765765765766,0.297560671428505)
--(axis cs:-0.75975975975976,0.298926971027289)
--(axis cs:-0.753753753753754,0.300288711976973)
--(axis cs:-0.747747747747748,0.301645775047225)
--(axis cs:-0.741741741741742,0.302998040974092)
--(axis cs:-0.735735735735736,0.304345390477095)
--(axis cs:-0.72972972972973,0.305687704276405)
--(axis cs:-0.723723723723724,0.307024863110094)
--(axis cs:-0.717717717717718,0.308356747751458)
--(axis cs:-0.711711711711712,0.309683239026412)
--(axis cs:-0.705705705705706,0.311004217830952)
--(axis cs:-0.6996996996997,0.312319565148675)
--(axis cs:-0.693693693693694,0.313629162068366)
--(axis cs:-0.687687687687688,0.314932889801632)
--(axis cs:-0.681681681681682,0.316230629700595)
--(axis cs:-0.675675675675676,0.317522263275628)
--(axis cs:-0.66966966966967,0.318807672213141)
--(axis cs:-0.663663663663664,0.320086738393407)
--(axis cs:-0.657657657657658,0.321359343908417)
--(axis cs:-0.651651651651652,0.322625371079786)
--(axis cs:-0.645645645645646,0.32388470247667)
--(axis cs:-0.63963963963964,0.325137220933723)
--(axis cs:-0.633633633633634,0.326382809569071)
--(axis cs:-0.627627627627628,0.327621351802301)
--(axis cs:-0.621621621621621,0.328852731372475)
--(axis cs:-0.615615615615616,0.330076832356145)
--(axis cs:-0.60960960960961,0.331293539185378)
--(axis cs:-0.603603603603604,0.332502736665788)
--(axis cs:-0.597597597597598,0.333704309994563)
--(axis cs:-0.591591591591591,0.334898144778487)
--(axis cs:-0.585585585585586,0.336084127051958)
--(axis cs:-0.57957957957958,0.337262143294989)
--(axis cs:-0.573573573573574,0.338432080451194)
--(axis cs:-0.567567567567568,0.339593825945755)
--(axis cs:-0.561561561561561,0.340747267703363)
--(axis cs:-0.555555555555555,0.341892294166129)
--(axis cs:-0.54954954954955,0.34302879431147)
--(axis cs:-0.543543543543544,0.344156657669946)
--(axis cs:-0.537537537537538,0.345275774343073)
--(axis cs:-0.531531531531531,0.346386035021075)
--(axis cs:-0.525525525525525,0.3474873310006)
--(axis cs:-0.51951951951952,0.34857955420238)
--(axis cs:-0.513513513513514,0.349662597188831)
--(axis cs:-0.507507507507508,0.350736353181597)
--(axis cs:-0.501501501501501,0.351800716079031)
--(axis cs:-0.495495495495495,0.352855580473602)
--(axis cs:-0.48948948948949,0.353900841669234)
--(axis cs:-0.483483483483484,0.35493639569857)
--(axis cs:-0.477477477477477,0.355962139340152)
--(axis cs:-0.471471471471471,0.356977970135516)
--(axis cs:-0.465465465465465,0.357983786406208)
--(axis cs:-0.45945945945946,0.358979487270697)
--(axis cs:-0.453453453453454,0.359964972661194)
--(axis cs:-0.447447447447447,0.360940143340381)
--(axis cs:-0.441441441441441,0.361904900918023)
--(axis cs:-0.435435435435435,0.36285914786748)
--(axis cs:-0.42942942942943,0.363802787542104)
--(axis cs:-0.423423423423424,0.364735724191528)
--(axis cs:-0.417417417417417,0.365657862977822)
--(axis cs:-0.411411411411411,0.36656910999154)
--(axis cs:-0.405405405405405,0.36746937226763)
--(axis cs:-0.3993993993994,0.368358557801224)
--(axis cs:-0.393393393393394,0.369236575563278)
--(axis cs:-0.387387387387387,0.370103335516092)
--(axis cs:-0.381381381381381,0.370958748628674)
--(axis cs:-0.375375375375375,0.371802726891965)
--(axis cs:-0.36936936936937,0.372635183333916)
--(axis cs:-0.363363363363364,0.373456032034409)
--(axis cs:-0.357357357357357,0.374265188140023)
--(axis cs:-0.351351351351351,0.375062567878638)
--(axis cs:-0.345345345345345,0.375848088573884)
--(axis cs:-0.33933933933934,0.376621668659409)
--(axis cs:-0.333333333333333,0.377383227692993)
--(axis cs:-0.327327327327327,0.378132686370474)
--(axis cs:-0.321321321321321,0.378869966539505)
--(axis cs:-0.315315315315315,0.379594991213131)
--(axis cs:-0.30930930930931,0.380307684583177)
--(axis cs:-0.303303303303303,0.381007972033453)
--(axis cs:-0.297297297297297,0.381695780152771)
--(axis cs:-0.291291291291291,0.382371036747761)
--(axis cs:-0.285285285285285,0.383033670855497)
--(axis cs:-0.279279279279279,0.383683612755922)
--(axis cs:-0.273273273273273,0.384320793984073)
--(axis cs:-0.267267267267267,0.38494514734209)
--(axis cs:-0.261261261261261,0.385556606911032)
--(axis cs:-0.255255255255255,0.386155108062468)
--(axis cs:-0.249249249249249,0.386740587469861)
--(axis cs:-0.243243243243243,0.387312983119735)
--(axis cs:-0.237237237237237,0.387872234322616)
--(axis cs:-0.231231231231231,0.388418281723758)
--(axis cs:-0.225225225225225,0.388951067313638)
--(axis cs:-0.219219219219219,0.389470534438229)
--(axis cs:-0.213213213213213,0.389976627809035)
--(axis cs:-0.207207207207207,0.390469293512899)
--(axis cs:-0.201201201201201,0.390948479021577)
--(axis cs:-0.195195195195195,0.391414133201066)
--(axis cs:-0.189189189189189,0.391866206320701)
--(axis cs:-0.183183183183183,0.392304650062005)
--(axis cs:-0.177177177177177,0.392729417527296)
--(axis cs:-0.171171171171171,0.393140463248047)
--(axis cs:-0.165165165165165,0.393537743192998)
--(axis cs:-0.159159159159159,0.393921214776013)
--(axis cs:-0.153153153153153,0.394290836863694)
--(axis cs:-0.147147147147147,0.39464656978273)
--(axis cs:-0.141141141141141,0.394988375326995)
--(axis cs:-0.135135135135135,0.395316216764386)
--(axis cs:-0.129129129129129,0.395630058843405)
--(axis cs:-0.123123123123123,0.395929867799471)
--(axis cs:-0.117117117117117,0.39621561136098)
--(axis cs:-0.111111111111111,0.396487258755088)
--(axis cs:-0.105105105105105,0.396744780713241)
--(axis cs:-0.099099099099099,0.396988149476425)
--(axis cs:-0.0930930930930933,0.397217338800158)
--(axis cs:-0.0870870870870872,0.397432323959201)
--(axis cs:-0.0810810810810811,0.39763308175201)
--(axis cs:-0.075075075075075,0.397819590504904)
--(axis cs:-0.069069069069069,0.397991830075968)
--(axis cs:-0.0630630630630633,0.398149781858677)
--(axis cs:-0.0570570570570572,0.398293428785244)
--(axis cs:-0.0510510510510511,0.3984227553297)
--(axis cs:-0.045045045045045,0.398537747510682)
--(axis cs:-0.0390390390390389,0.39863839289396)
--(axis cs:-0.0330330330330328,0.398724680594675)
--(axis cs:-0.0270270270270272,0.398796601279297)
--(axis cs:-0.0210210210210211,0.398854147167316)
--(axis cs:-0.015015015015015,0.398897312032637)
--(axis cs:-0.00900900900900892,0.398926091204706)
--(axis cs:-0.00300300300300282,0.398940481569353)
--(axis cs:0.00300300300300282,0.398940481569353)
--(axis cs:0.00900900900900892,0.398926091204706)
--(axis cs:0.015015015015015,0.398897312032637)
--(axis cs:0.0210210210210211,0.398854147167316)
--(axis cs:0.0270270270270272,0.398796601279297)
--(axis cs:0.0330330330330328,0.398724680594675)
--(axis cs:0.0390390390390389,0.39863839289396)
--(axis cs:0.045045045045045,0.398537747510682)
--(axis cs:0.0510510510510511,0.3984227553297)
--(axis cs:0.0570570570570572,0.398293428785244)
--(axis cs:0.0630630630630629,0.398149781858677)
--(axis cs:0.069069069069069,0.397991830075968)
--(axis cs:0.075075075075075,0.397819590504904)
--(axis cs:0.0810810810810811,0.39763308175201)
--(axis cs:0.0870870870870872,0.397432323959201)
--(axis cs:0.0930930930930929,0.397217338800158)
--(axis cs:0.099099099099099,0.396988149476425)
--(axis cs:0.105105105105105,0.396744780713241)
--(axis cs:0.111111111111111,0.396487258755088)
--(axis cs:0.117117117117117,0.39621561136098)
--(axis cs:0.123123123123123,0.395929867799471)
--(axis cs:0.129129129129129,0.395630058843405)
--(axis cs:0.135135135135135,0.395316216764386)
--(axis cs:0.141141141141141,0.394988375326995)
--(axis cs:0.147147147147147,0.39464656978273)
--(axis cs:0.153153153153153,0.394290836863694)
--(axis cs:0.159159159159159,0.393921214776013)
--(axis cs:0.165165165165165,0.393537743192998)
--(axis cs:0.171171171171171,0.393140463248047)
--(axis cs:0.177177177177177,0.392729417527296)
--(axis cs:0.183183183183183,0.392304650062005)
--(axis cs:0.189189189189189,0.391866206320701)
--(axis cs:0.195195195195195,0.391414133201066)
--(axis cs:0.201201201201201,0.390948479021577)
--(axis cs:0.207207207207207,0.390469293512899)
--(axis cs:0.213213213213213,0.389976627809035)
--(axis cs:0.219219219219219,0.389470534438229)
--(axis cs:0.225225225225225,0.388951067313638)
--(axis cs:0.231231231231231,0.388418281723758)
--(axis cs:0.237237237237237,0.387872234322616)
--(axis cs:0.243243243243243,0.387312983119735)
--(axis cs:0.249249249249249,0.386740587469861)
--(axis cs:0.255255255255255,0.386155108062468)
--(axis cs:0.261261261261261,0.385556606911032)
--(axis cs:0.267267267267267,0.38494514734209)
--(axis cs:0.273273273273273,0.384320793984073)
--(axis cs:0.279279279279279,0.383683612755922)
--(axis cs:0.285285285285285,0.383033670855497)
--(axis cs:0.291291291291291,0.382371036747761)
--(axis cs:0.297297297297297,0.381695780152771)
--(axis cs:0.303303303303303,0.381007972033453)
--(axis cs:0.309309309309309,0.380307684583177)
--(axis cs:0.315315315315315,0.379594991213131)
--(axis cs:0.321321321321321,0.378869966539505)
--(axis cs:0.327327327327327,0.378132686370474)
--(axis cs:0.333333333333333,0.377383227692993)
--(axis cs:0.339339339339339,0.376621668659409)
--(axis cs:0.345345345345345,0.375848088573884)
--(axis cs:0.351351351351351,0.375062567878638)
--(axis cs:0.357357357357357,0.374265188140023)
--(axis cs:0.363363363363364,0.373456032034409)
--(axis cs:0.369369369369369,0.372635183333916)
--(axis cs:0.375375375375375,0.371802726891965)
--(axis cs:0.381381381381381,0.370958748628674)
--(axis cs:0.387387387387387,0.370103335516092)
--(axis cs:0.393393393393394,0.369236575563278)
--(axis cs:0.399399399399399,0.368358557801224)
--(axis cs:0.405405405405405,0.36746937226763)
--(axis cs:0.411411411411411,0.36656910999154)
--(axis cs:0.417417417417417,0.365657862977822)
--(axis cs:0.423423423423424,0.364735724191528)
--(axis cs:0.429429429429429,0.363802787542104)
--(axis cs:0.435435435435435,0.36285914786748)
--(axis cs:0.441441441441441,0.361904900918023)
--(axis cs:0.447447447447447,0.360940143340381)
--(axis cs:0.453453453453454,0.359964972661194)
--(axis cs:0.45945945945946,0.358979487270697)
--(axis cs:0.465465465465465,0.357983786406208)
--(axis cs:0.471471471471471,0.356977970135516)
--(axis cs:0.477477477477477,0.355962139340152)
--(axis cs:0.483483483483484,0.35493639569857)
--(axis cs:0.48948948948949,0.353900841669234)
--(axis cs:0.495495495495495,0.352855580473602)
--(axis cs:0.501501501501501,0.351800716079031)
--(axis cs:0.507507507507508,0.350736353181597)
--(axis cs:0.513513513513514,0.349662597188831)
--(axis cs:0.51951951951952,0.34857955420238)
--(axis cs:0.525525525525525,0.3474873310006)
--(axis cs:0.531531531531531,0.346386035021075)
--(axis cs:0.537537537537538,0.345275774343073)
--(axis cs:0.543543543543544,0.344156657669946)
--(axis cs:0.54954954954955,0.34302879431147)
--(axis cs:0.555555555555555,0.341892294166129)
--(axis cs:0.561561561561561,0.340747267703363)
--(axis cs:0.567567567567568,0.339593825945755)
--(axis cs:0.573573573573574,0.338432080451194)
--(axis cs:0.57957957957958,0.337262143294989)
--(axis cs:0.585585585585585,0.336084127051958)
--(axis cs:0.591591591591591,0.334898144778487)
--(axis cs:0.597597597597598,0.333704309994563)
--(axis cs:0.603603603603604,0.332502736665788)
--(axis cs:0.60960960960961,0.331293539185378)
--(axis cs:0.615615615615615,0.330076832356145)
--(axis cs:0.621621621621621,0.328852731372475)
--(axis cs:0.627627627627628,0.327621351802301)
--(axis cs:0.633633633633634,0.326382809569071)
--(axis cs:0.63963963963964,0.325137220933723)
--(axis cs:0.645645645645645,0.32388470247667)
--(axis cs:0.651651651651652,0.322625371079786)
--(axis cs:0.657657657657658,0.321359343908417)
--(axis cs:0.663663663663664,0.320086738393407)
--(axis cs:0.66966966966967,0.318807672213141)
--(axis cs:0.675675675675675,0.317522263275628)
--(axis cs:0.681681681681682,0.316230629700595)
--(axis cs:0.687687687687688,0.314932889801632)
--(axis cs:0.693693693693694,0.313629162068366)
--(axis cs:0.6996996996997,0.312319565148675)
--(axis cs:0.705705705705705,0.311004217830952)
--(axis cs:0.711711711711712,0.309683239026412)
--(axis cs:0.717717717717718,0.308356747751458)
--(axis cs:0.723723723723724,0.307024863110094)
--(axis cs:0.72972972972973,0.305687704276405)
--(axis cs:0.735735735735736,0.304345390477095)
--(axis cs:0.741741741741742,0.302998040974092)
--(axis cs:0.747747747747748,0.301645775047225)
--(axis cs:0.753753753753754,0.300288711976973)
--(axis cs:0.75975975975976,0.298926971027289)
--(axis cs:0.765765765765766,0.297560671428505)
--(axis cs:0.771771771771772,0.29618993236032)
--(axis cs:0.777777777777778,0.294814872934877)
--(axis cs:0.783783783783784,0.293435612179928)
--(axis cs:0.78978978978979,0.292052269022087)
--(axis cs:0.795795795795796,0.29066496227019)
--(axis cs:0.801801801801802,0.289273810598746)
--(axis cs:0.807807807807808,0.287878932531491)
--(axis cs:0.813813813813814,0.286480446425052)
--(axis cs:0.81981981981982,0.285078470452718)
--(axis cs:0.825825825825826,0.283673122588316)
--(axis cs:0.831831831831832,0.282264520590213)
--(axis cs:0.837837837837838,0.280852781985427)
--(axis cs:0.843843843843844,0.279438024053859)
--(axis cs:0.84984984984985,0.278020363812654)
--(axis cs:0.855855855855856,0.276599918000677)
--(axis cs:0.861861861861862,0.275176803063131)
--(axis cs:0.867867867867868,0.273751135136292)
--(axis cs:0.873873873873874,0.272323030032391)
--(axis cs:0.87987987987988,0.270892603224623)
--(axis cs:0.885885885885886,0.269459969832298)
--(axis cs:0.891891891891892,0.268025244606136)
--(axis cs:0.897897897897898,0.266588541913699)
--(axis cs:0.903903903903904,0.265149975724979)
--(axis cs:0.90990990990991,0.263709659598126)
--(axis cs:0.915915915915916,0.262267706665329)
--(axis cs:0.921921921921922,0.260824229618852)
--(axis cs:0.927927927927928,0.259379340697227)
--(axis cs:0.933933933933934,0.257933151671597)
--(axis cs:0.93993993993994,0.256485773832225)
--(axis cs:0.945945945945946,0.255037317975163)
--(axis cs:0.951951951951952,0.253587894389084)
--(axis cs:0.957957957957958,0.252137612842278)
--(axis cs:0.963963963963964,0.250686582569819)
--(axis cs:0.96996996996997,0.249234912260894)
--(axis cs:0.975975975975976,0.247782710046314)
--(axis cs:0.981981981981982,0.246330083486184)
--(axis cs:0.987987987987988,0.244877139557762)
--(axis cs:0.993993993993994,0.243423984643478)
--(axis cs:0.993993993993994,0)
--(axis cs:0.993993993993994,0)
--(axis cs:0.987987987987988,0)
--(axis cs:0.981981981981982,0)
--(axis cs:0.975975975975976,0)
--(axis cs:0.96996996996997,0)
--(axis cs:0.963963963963964,0)
--(axis cs:0.957957957957958,0)
--(axis cs:0.951951951951952,0)
--(axis cs:0.945945945945946,0)
--(axis cs:0.93993993993994,0)
--(axis cs:0.933933933933934,0)
--(axis cs:0.927927927927928,0)
--(axis cs:0.921921921921922,0)
--(axis cs:0.915915915915916,0)
--(axis cs:0.90990990990991,0)
--(axis cs:0.903903903903904,0)
--(axis cs:0.897897897897898,0)
--(axis cs:0.891891891891892,0)
--(axis cs:0.885885885885886,0)
--(axis cs:0.87987987987988,0)
--(axis cs:0.873873873873874,0)
--(axis cs:0.867867867867868,0)
--(axis cs:0.861861861861862,0)
--(axis cs:0.855855855855856,0)
--(axis cs:0.84984984984985,0)
--(axis cs:0.843843843843844,0)
--(axis cs:0.837837837837838,0)
--(axis cs:0.831831831831832,0)
--(axis cs:0.825825825825826,0)
--(axis cs:0.81981981981982,0)
--(axis cs:0.813813813813814,0)
--(axis cs:0.807807807807808,0)
--(axis cs:0.801801801801802,0)
--(axis cs:0.795795795795796,0)
--(axis cs:0.78978978978979,0)
--(axis cs:0.783783783783784,0)
--(axis cs:0.777777777777778,0)
--(axis cs:0.771771771771772,0)
--(axis cs:0.765765765765766,0)
--(axis cs:0.75975975975976,0)
--(axis cs:0.753753753753754,0)
--(axis cs:0.747747747747748,0)
--(axis cs:0.741741741741742,0)
--(axis cs:0.735735735735736,0)
--(axis cs:0.72972972972973,0)
--(axis cs:0.723723723723724,0)
--(axis cs:0.717717717717718,0)
--(axis cs:0.711711711711712,0)
--(axis cs:0.705705705705705,0)
--(axis cs:0.6996996996997,0)
--(axis cs:0.693693693693694,0)
--(axis cs:0.687687687687688,0)
--(axis cs:0.681681681681682,0)
--(axis cs:0.675675675675675,0)
--(axis cs:0.66966966966967,0)
--(axis cs:0.663663663663664,0)
--(axis cs:0.657657657657658,0)
--(axis cs:0.651651651651652,0)
--(axis cs:0.645645645645645,0)
--(axis cs:0.63963963963964,0)
--(axis cs:0.633633633633634,0)
--(axis cs:0.627627627627628,0)
--(axis cs:0.621621621621621,0)
--(axis cs:0.615615615615615,0)
--(axis cs:0.60960960960961,0)
--(axis cs:0.603603603603604,0)
--(axis cs:0.597597597597598,0)
--(axis cs:0.591591591591591,0)
--(axis cs:0.585585585585585,0)
--(axis cs:0.57957957957958,0)
--(axis cs:0.573573573573574,0)
--(axis cs:0.567567567567568,0)
--(axis cs:0.561561561561561,0)
--(axis cs:0.555555555555555,0)
--(axis cs:0.54954954954955,0)
--(axis cs:0.543543543543544,0)
--(axis cs:0.537537537537538,0)
--(axis cs:0.531531531531531,0)
--(axis cs:0.525525525525525,0)
--(axis cs:0.51951951951952,0)
--(axis cs:0.513513513513514,0)
--(axis cs:0.507507507507508,0)
--(axis cs:0.501501501501501,0)
--(axis cs:0.495495495495495,0)
--(axis cs:0.48948948948949,0)
--(axis cs:0.483483483483484,0)
--(axis cs:0.477477477477477,0)
--(axis cs:0.471471471471471,0)
--(axis cs:0.465465465465465,0)
--(axis cs:0.45945945945946,0)
--(axis cs:0.453453453453454,0)
--(axis cs:0.447447447447447,0)
--(axis cs:0.441441441441441,0)
--(axis cs:0.435435435435435,0)
--(axis cs:0.429429429429429,0)
--(axis cs:0.423423423423424,0)
--(axis cs:0.417417417417417,0)
--(axis cs:0.411411411411411,0)
--(axis cs:0.405405405405405,0)
--(axis cs:0.399399399399399,0)
--(axis cs:0.393393393393394,0)
--(axis cs:0.387387387387387,0)
--(axis cs:0.381381381381381,0)
--(axis cs:0.375375375375375,0)
--(axis cs:0.369369369369369,0)
--(axis cs:0.363363363363364,0)
--(axis cs:0.357357357357357,0)
--(axis cs:0.351351351351351,0)
--(axis cs:0.345345345345345,0)
--(axis cs:0.339339339339339,0)
--(axis cs:0.333333333333333,0)
--(axis cs:0.327327327327327,0)
--(axis cs:0.321321321321321,0)
--(axis cs:0.315315315315315,0)
--(axis cs:0.309309309309309,0)
--(axis cs:0.303303303303303,0)
--(axis cs:0.297297297297297,0)
--(axis cs:0.291291291291291,0)
--(axis cs:0.285285285285285,0)
--(axis cs:0.279279279279279,0)
--(axis cs:0.273273273273273,0)
--(axis cs:0.267267267267267,0)
--(axis cs:0.261261261261261,0)
--(axis cs:0.255255255255255,0)
--(axis cs:0.249249249249249,0)
--(axis cs:0.243243243243243,0)
--(axis cs:0.237237237237237,0)
--(axis cs:0.231231231231231,0)
--(axis cs:0.225225225225225,0)
--(axis cs:0.219219219219219,0)
--(axis cs:0.213213213213213,0)
--(axis cs:0.207207207207207,0)
--(axis cs:0.201201201201201,0)
--(axis cs:0.195195195195195,0)
--(axis cs:0.189189189189189,0)
--(axis cs:0.183183183183183,0)
--(axis cs:0.177177177177177,0)
--(axis cs:0.171171171171171,0)
--(axis cs:0.165165165165165,0)
--(axis cs:0.159159159159159,0)
--(axis cs:0.153153153153153,0)
--(axis cs:0.147147147147147,0)
--(axis cs:0.141141141141141,0)
--(axis cs:0.135135135135135,0)
--(axis cs:0.129129129129129,0)
--(axis cs:0.123123123123123,0)
--(axis cs:0.117117117117117,0)
--(axis cs:0.111111111111111,0)
--(axis cs:0.105105105105105,0)
--(axis cs:0.099099099099099,0)
--(axis cs:0.0930930930930929,0)
--(axis cs:0.0870870870870872,0)
--(axis cs:0.0810810810810811,0)
--(axis cs:0.075075075075075,0)
--(axis cs:0.069069069069069,0)
--(axis cs:0.0630630630630629,0)
--(axis cs:0.0570570570570572,0)
--(axis cs:0.0510510510510511,0)
--(axis cs:0.045045045045045,0)
--(axis cs:0.0390390390390389,0)
--(axis cs:0.0330330330330328,0)
--(axis cs:0.0270270270270272,0)
--(axis cs:0.0210210210210211,0)
--(axis cs:0.015015015015015,0)
--(axis cs:0.00900900900900892,0)
--(axis cs:0.00300300300300282,0)
--(axis cs:-0.00300300300300282,0)
--(axis cs:-0.00900900900900892,0)
--(axis cs:-0.015015015015015,0)
--(axis cs:-0.0210210210210211,0)
--(axis cs:-0.0270270270270272,0)
--(axis cs:-0.0330330330330328,0)
--(axis cs:-0.0390390390390389,0)
--(axis cs:-0.045045045045045,0)
--(axis cs:-0.0510510510510511,0)
--(axis cs:-0.0570570570570572,0)
--(axis cs:-0.0630630630630633,0)
--(axis cs:-0.069069069069069,0)
--(axis cs:-0.075075075075075,0)
--(axis cs:-0.0810810810810811,0)
--(axis cs:-0.0870870870870872,0)
--(axis cs:-0.0930930930930933,0)
--(axis cs:-0.099099099099099,0)
--(axis cs:-0.105105105105105,0)
--(axis cs:-0.111111111111111,0)
--(axis cs:-0.117117117117117,0)
--(axis cs:-0.123123123123123,0)
--(axis cs:-0.129129129129129,0)
--(axis cs:-0.135135135135135,0)
--(axis cs:-0.141141141141141,0)
--(axis cs:-0.147147147147147,0)
--(axis cs:-0.153153153153153,0)
--(axis cs:-0.159159159159159,0)
--(axis cs:-0.165165165165165,0)
--(axis cs:-0.171171171171171,0)
--(axis cs:-0.177177177177177,0)
--(axis cs:-0.183183183183183,0)
--(axis cs:-0.189189189189189,0)
--(axis cs:-0.195195195195195,0)
--(axis cs:-0.201201201201201,0)
--(axis cs:-0.207207207207207,0)
--(axis cs:-0.213213213213213,0)
--(axis cs:-0.219219219219219,0)
--(axis cs:-0.225225225225225,0)
--(axis cs:-0.231231231231231,0)
--(axis cs:-0.237237237237237,0)
--(axis cs:-0.243243243243243,0)
--(axis cs:-0.249249249249249,0)
--(axis cs:-0.255255255255255,0)
--(axis cs:-0.261261261261261,0)
--(axis cs:-0.267267267267267,0)
--(axis cs:-0.273273273273273,0)
--(axis cs:-0.279279279279279,0)
--(axis cs:-0.285285285285285,0)
--(axis cs:-0.291291291291291,0)
--(axis cs:-0.297297297297297,0)
--(axis cs:-0.303303303303303,0)
--(axis cs:-0.30930930930931,0)
--(axis cs:-0.315315315315315,0)
--(axis cs:-0.321321321321321,0)
--(axis cs:-0.327327327327327,0)
--(axis cs:-0.333333333333333,0)
--(axis cs:-0.33933933933934,0)
--(axis cs:-0.345345345345345,0)
--(axis cs:-0.351351351351351,0)
--(axis cs:-0.357357357357357,0)
--(axis cs:-0.363363363363364,0)
--(axis cs:-0.36936936936937,0)
--(axis cs:-0.375375375375375,0)
--(axis cs:-0.381381381381381,0)
--(axis cs:-0.387387387387387,0)
--(axis cs:-0.393393393393394,0)
--(axis cs:-0.3993993993994,0)
--(axis cs:-0.405405405405405,0)
--(axis cs:-0.411411411411411,0)
--(axis cs:-0.417417417417417,0)
--(axis cs:-0.423423423423424,0)
--(axis cs:-0.42942942942943,0)
--(axis cs:-0.435435435435435,0)
--(axis cs:-0.441441441441441,0)
--(axis cs:-0.447447447447447,0)
--(axis cs:-0.453453453453454,0)
--(axis cs:-0.45945945945946,0)
--(axis cs:-0.465465465465465,0)
--(axis cs:-0.471471471471471,0)
--(axis cs:-0.477477477477477,0)
--(axis cs:-0.483483483483484,0)
--(axis cs:-0.48948948948949,0)
--(axis cs:-0.495495495495495,0)
--(axis cs:-0.501501501501501,0)
--(axis cs:-0.507507507507508,0)
--(axis cs:-0.513513513513514,0)
--(axis cs:-0.51951951951952,0)
--(axis cs:-0.525525525525525,0)
--(axis cs:-0.531531531531531,0)
--(axis cs:-0.537537537537538,0)
--(axis cs:-0.543543543543544,0)
--(axis cs:-0.54954954954955,0)
--(axis cs:-0.555555555555555,0)
--(axis cs:-0.561561561561561,0)
--(axis cs:-0.567567567567568,0)
--(axis cs:-0.573573573573574,0)
--(axis cs:-0.57957957957958,0)
--(axis cs:-0.585585585585586,0)
--(axis cs:-0.591591591591591,0)
--(axis cs:-0.597597597597598,0)
--(axis cs:-0.603603603603604,0)
--(axis cs:-0.60960960960961,0)
--(axis cs:-0.615615615615616,0)
--(axis cs:-0.621621621621621,0)
--(axis cs:-0.627627627627628,0)
--(axis cs:-0.633633633633634,0)
--(axis cs:-0.63963963963964,0)
--(axis cs:-0.645645645645646,0)
--(axis cs:-0.651651651651652,0)
--(axis cs:-0.657657657657658,0)
--(axis cs:-0.663663663663664,0)
--(axis cs:-0.66966966966967,0)
--(axis cs:-0.675675675675676,0)
--(axis cs:-0.681681681681682,0)
--(axis cs:-0.687687687687688,0)
--(axis cs:-0.693693693693694,0)
--(axis cs:-0.6996996996997,0)
--(axis cs:-0.705705705705706,0)
--(axis cs:-0.711711711711712,0)
--(axis cs:-0.717717717717718,0)
--(axis cs:-0.723723723723724,0)
--(axis cs:-0.72972972972973,0)
--(axis cs:-0.735735735735736,0)
--(axis cs:-0.741741741741742,0)
--(axis cs:-0.747747747747748,0)
--(axis cs:-0.753753753753754,0)
--(axis cs:-0.75975975975976,0)
--(axis cs:-0.765765765765766,0)
--(axis cs:-0.771771771771772,0)
--(axis cs:-0.777777777777778,0)
--(axis cs:-0.783783783783784,0)
--(axis cs:-0.78978978978979,0)
--(axis cs:-0.795795795795796,0)
--(axis cs:-0.801801801801802,0)
--(axis cs:-0.807807807807808,0)
--(axis cs:-0.813813813813814,0)
--(axis cs:-0.81981981981982,0)
--(axis cs:-0.825825825825826,0)
--(axis cs:-0.831831831831832,0)
--(axis cs:-0.837837837837838,0)
--(axis cs:-0.843843843843844,0)
--(axis cs:-0.84984984984985,0)
--(axis cs:-0.855855855855856,0)
--(axis cs:-0.861861861861862,0)
--(axis cs:-0.867867867867868,0)
--(axis cs:-0.873873873873874,0)
--(axis cs:-0.87987987987988,0)
--(axis cs:-0.885885885885886,0)
--(axis cs:-0.891891891891892,0)
--(axis cs:-0.897897897897898,0)
--(axis cs:-0.903903903903904,0)
--(axis cs:-0.90990990990991,0)
--(axis cs:-0.915915915915916,0)
--(axis cs:-0.921921921921922,0)
--(axis cs:-0.927927927927928,0)
--(axis cs:-0.933933933933934,0)
--(axis cs:-0.93993993993994,0)
--(axis cs:-0.945945945945946,0)
--(axis cs:-0.951951951951952,0)
--(axis cs:-0.957957957957958,0)
--(axis cs:-0.963963963963964,0)
--(axis cs:-0.96996996996997,0)
--(axis cs:-0.975975975975976,0)
--(axis cs:-0.981981981981982,0)
--(axis cs:-0.987987987987988,0)
--(axis cs:-0.993993993993994,0)
--(axis cs:-1,0)
--(axis cs:-1.00600600600601,0)
--(axis cs:-1.01201201201201,0)
--(axis cs:-1.01801801801802,0)
--(axis cs:-1.02402402402402,0)
--(axis cs:-1.03003003003003,0)
--(axis cs:-1.03603603603604,0)
--(axis cs:-1.04204204204204,0)
--(axis cs:-1.04804804804805,0)
--(axis cs:-1.05405405405405,0)
--(axis cs:-1.06006006006006,0)
--(axis cs:-1.06606606606607,0)
--(axis cs:-1.07207207207207,0)
--(axis cs:-1.07807807807808,0)
--(axis cs:-1.08408408408408,0)
--(axis cs:-1.09009009009009,0)
--(axis cs:-1.0960960960961,0)
--(axis cs:-1.1021021021021,0)
--(axis cs:-1.10810810810811,0)
--(axis cs:-1.11411411411411,0)
--(axis cs:-1.12012012012012,0)
--(axis cs:-1.12612612612613,0)
--(axis cs:-1.13213213213213,0)
--(axis cs:-1.13813813813814,0)
--(axis cs:-1.14414414414414,0)
--(axis cs:-1.15015015015015,0)
--(axis cs:-1.15615615615616,0)
--(axis cs:-1.16216216216216,0)
--(axis cs:-1.16816816816817,0)
--(axis cs:-1.17417417417417,0)
--(axis cs:-1.18018018018018,0)
--(axis cs:-1.18618618618619,0)
--(axis cs:-1.19219219219219,0)
--(axis cs:-1.1981981981982,0)
--(axis cs:-1.2042042042042,0)
--(axis cs:-1.21021021021021,0)
--(axis cs:-1.21621621621622,0)
--(axis cs:-1.22222222222222,0)
--(axis cs:-1.22822822822823,0)
--(axis cs:-1.23423423423423,0)
--(axis cs:-1.24024024024024,0)
--(axis cs:-1.24624624624625,0)
--(axis cs:-1.25225225225225,0)
--(axis cs:-1.25825825825826,0)
--(axis cs:-1.26426426426426,0)
--(axis cs:-1.27027027027027,0)
--(axis cs:-1.27627627627628,0)
--(axis cs:-1.28228228228228,0)
--(axis cs:-1.28828828828829,0)
--(axis cs:-1.29429429429429,0)
--(axis cs:-1.3003003003003,0)
--(axis cs:-1.30630630630631,0)
--(axis cs:-1.31231231231231,0)
--(axis cs:-1.31831831831832,0)
--(axis cs:-1.32432432432432,0)
--(axis cs:-1.33033033033033,0)
--(axis cs:-1.33633633633634,0)
--(axis cs:-1.34234234234234,0)
--(axis cs:-1.34834834834835,0)
--(axis cs:-1.35435435435435,0)
--(axis cs:-1.36036036036036,0)
--(axis cs:-1.36636636636637,0)
--(axis cs:-1.37237237237237,0)
--(axis cs:-1.37837837837838,0)
--(axis cs:-1.38438438438438,0)
--(axis cs:-1.39039039039039,0)
--(axis cs:-1.3963963963964,0)
--(axis cs:-1.4024024024024,0)
--(axis cs:-1.40840840840841,0)
--(axis cs:-1.41441441441441,0)
--(axis cs:-1.42042042042042,0)
--(axis cs:-1.42642642642643,0)
--(axis cs:-1.43243243243243,0)
--(axis cs:-1.43843843843844,0)
--(axis cs:-1.44444444444444,0)
--(axis cs:-1.45045045045045,0)
--(axis cs:-1.45645645645646,0)
--(axis cs:-1.46246246246246,0)
--(axis cs:-1.46846846846847,0)
--(axis cs:-1.47447447447447,0)
--(axis cs:-1.48048048048048,0)
--(axis cs:-1.48648648648649,0)
--(axis cs:-1.49249249249249,0)
--(axis cs:-1.4984984984985,0)
--(axis cs:-1.5045045045045,0)
--(axis cs:-1.51051051051051,0)
--(axis cs:-1.51651651651652,0)
--(axis cs:-1.52252252252252,0)
--(axis cs:-1.52852852852853,0)
--(axis cs:-1.53453453453453,0)
--(axis cs:-1.54054054054054,0)
--(axis cs:-1.54654654654655,0)
--(axis cs:-1.55255255255255,0)
--(axis cs:-1.55855855855856,0)
--(axis cs:-1.56456456456456,0)
--(axis cs:-1.57057057057057,0)
--(axis cs:-1.57657657657658,0)
--(axis cs:-1.58258258258258,0)
--(axis cs:-1.58858858858859,0)
--(axis cs:-1.59459459459459,0)
--(axis cs:-1.6006006006006,0)
--(axis cs:-1.60660660660661,0)
--(axis cs:-1.61261261261261,0)
--(axis cs:-1.61861861861862,0)
--(axis cs:-1.62462462462462,0)
--(axis cs:-1.63063063063063,0)
--(axis cs:-1.63663663663664,0)
--(axis cs:-1.64264264264264,0)
--(axis cs:-1.64864864864865,0)
--(axis cs:-1.65465465465465,0)
--(axis cs:-1.66066066066066,0)
--(axis cs:-1.66666666666667,0)
--(axis cs:-1.67267267267267,0)
--(axis cs:-1.67867867867868,0)
--(axis cs:-1.68468468468468,0)
--(axis cs:-1.69069069069069,0)
--(axis cs:-1.6966966966967,0)
--(axis cs:-1.7027027027027,0)
--(axis cs:-1.70870870870871,0)
--(axis cs:-1.71471471471471,0)
--(axis cs:-1.72072072072072,0)
--(axis cs:-1.72672672672673,0)
--(axis cs:-1.73273273273273,0)
--(axis cs:-1.73873873873874,0)
--(axis cs:-1.74474474474474,0)
--(axis cs:-1.75075075075075,0)
--(axis cs:-1.75675675675676,0)
--(axis cs:-1.76276276276276,0)
--(axis cs:-1.76876876876877,0)
--(axis cs:-1.77477477477477,0)
--(axis cs:-1.78078078078078,0)
--(axis cs:-1.78678678678679,0)
--(axis cs:-1.79279279279279,0)
--(axis cs:-1.7987987987988,0)
--(axis cs:-1.8048048048048,0)
--(axis cs:-1.81081081081081,0)
--(axis cs:-1.81681681681682,0)
--(axis cs:-1.82282282282282,0)
--(axis cs:-1.82882882882883,0)
--(axis cs:-1.83483483483483,0)
--(axis cs:-1.84084084084084,0)
--(axis cs:-1.84684684684685,0)
--(axis cs:-1.85285285285285,0)
--(axis cs:-1.85885885885886,0)
--(axis cs:-1.86486486486486,0)
--(axis cs:-1.87087087087087,0)
--(axis cs:-1.87687687687688,0)
--(axis cs:-1.88288288288288,0)
--(axis cs:-1.88888888888889,0)
--(axis cs:-1.89489489489489,0)
--(axis cs:-1.9009009009009,0)
--(axis cs:-1.90690690690691,0)
--(axis cs:-1.91291291291291,0)
--(axis cs:-1.91891891891892,0)
--(axis cs:-1.92492492492492,0)
--(axis cs:-1.93093093093093,0)
--(axis cs:-1.93693693693694,0)
--(axis cs:-1.94294294294294,0)
--(axis cs:-1.94894894894895,0)
--(axis cs:-1.95495495495495,0)
--(axis cs:-1.96096096096096,0)
--(axis cs:-1.96696696696697,0)
--(axis cs:-1.97297297297297,0)
--(axis cs:-1.97897897897898,0)
--(axis cs:-1.98498498498498,0)
--(axis cs:-1.99099099099099,0)
--(axis cs:-1.996996996997,0)
--(axis cs:-2.003003003003,0)
--(axis cs:-2.00900900900901,0)
--(axis cs:-2.01501501501502,0)
--(axis cs:-2.02102102102102,0)
--(axis cs:-2.02702702702703,0)
--(axis cs:-2.03303303303303,0)
--(axis cs:-2.03903903903904,0)
--(axis cs:-2.04504504504505,0)
--(axis cs:-2.05105105105105,0)
--(axis cs:-2.05705705705706,0)
--(axis cs:-2.06306306306306,0)
--(axis cs:-2.06906906906907,0)
--(axis cs:-2.07507507507508,0)
--(axis cs:-2.08108108108108,0)
--(axis cs:-2.08708708708709,0)
--(axis cs:-2.09309309309309,0)
--(axis cs:-2.0990990990991,0)
--(axis cs:-2.10510510510511,0)
--(axis cs:-2.11111111111111,0)
--(axis cs:-2.11711711711712,0)
--(axis cs:-2.12312312312312,0)
--(axis cs:-2.12912912912913,0)
--(axis cs:-2.13513513513514,0)
--(axis cs:-2.14114114114114,0)
--(axis cs:-2.14714714714715,0)
--(axis cs:-2.15315315315315,0)
--(axis cs:-2.15915915915916,0)
--(axis cs:-2.16516516516517,0)
--(axis cs:-2.17117117117117,0)
--(axis cs:-2.17717717717718,0)
--(axis cs:-2.18318318318318,0)
--(axis cs:-2.18918918918919,0)
--(axis cs:-2.1951951951952,0)
--(axis cs:-2.2012012012012,0)
--(axis cs:-2.20720720720721,0)
--(axis cs:-2.21321321321321,0)
--(axis cs:-2.21921921921922,0)
--(axis cs:-2.22522522522523,0)
--(axis cs:-2.23123123123123,0)
--(axis cs:-2.23723723723724,0)
--(axis cs:-2.24324324324324,0)
--(axis cs:-2.24924924924925,0)
--(axis cs:-2.25525525525526,0)
--(axis cs:-2.26126126126126,0)
--(axis cs:-2.26726726726727,0)
--(axis cs:-2.27327327327327,0)
--(axis cs:-2.27927927927928,0)
--(axis cs:-2.28528528528529,0)
--(axis cs:-2.29129129129129,0)
--(axis cs:-2.2972972972973,0)
--(axis cs:-2.3033033033033,0)
--(axis cs:-2.30930930930931,0)
--(axis cs:-2.31531531531532,0)
--(axis cs:-2.32132132132132,0)
--(axis cs:-2.32732732732733,0)
--(axis cs:-2.33333333333333,0)
--(axis cs:-2.33933933933934,0)
--(axis cs:-2.34534534534535,0)
--(axis cs:-2.35135135135135,0)
--(axis cs:-2.35735735735736,0)
--(axis cs:-2.36336336336336,0)
--(axis cs:-2.36936936936937,0)
--(axis cs:-2.37537537537538,0)
--(axis cs:-2.38138138138138,0)
--(axis cs:-2.38738738738739,0)
--(axis cs:-2.39339339339339,0)
--(axis cs:-2.3993993993994,0)
--(axis cs:-2.40540540540541,0)
--(axis cs:-2.41141141141141,0)
--(axis cs:-2.41741741741742,0)
--(axis cs:-2.42342342342342,0)
--(axis cs:-2.42942942942943,0)
--(axis cs:-2.43543543543544,0)
--(axis cs:-2.44144144144144,0)
--(axis cs:-2.44744744744745,0)
--(axis cs:-2.45345345345345,0)
--(axis cs:-2.45945945945946,0)
--(axis cs:-2.46546546546547,0)
--(axis cs:-2.47147147147147,0)
--(axis cs:-2.47747747747748,0)
--(axis cs:-2.48348348348348,0)
--(axis cs:-2.48948948948949,0)
--(axis cs:-2.4954954954955,0)
--(axis cs:-2.5015015015015,0)
--(axis cs:-2.50750750750751,0)
--(axis cs:-2.51351351351351,0)
--(axis cs:-2.51951951951952,0)
--(axis cs:-2.52552552552553,0)
--(axis cs:-2.53153153153153,0)
--(axis cs:-2.53753753753754,0)
--(axis cs:-2.54354354354354,0)
--(axis cs:-2.54954954954955,0)
--(axis cs:-2.55555555555556,0)
--(axis cs:-2.56156156156156,0)
--(axis cs:-2.56756756756757,0)
--(axis cs:-2.57357357357357,0)
--(axis cs:-2.57957957957958,0)
--(axis cs:-2.58558558558559,0)
--(axis cs:-2.59159159159159,0)
--(axis cs:-2.5975975975976,0)
--(axis cs:-2.6036036036036,0)
--(axis cs:-2.60960960960961,0)
--(axis cs:-2.61561561561562,0)
--(axis cs:-2.62162162162162,0)
--(axis cs:-2.62762762762763,0)
--(axis cs:-2.63363363363363,0)
--(axis cs:-2.63963963963964,0)
--(axis cs:-2.64564564564565,0)
--(axis cs:-2.65165165165165,0)
--(axis cs:-2.65765765765766,0)
--(axis cs:-2.66366366366366,0)
--(axis cs:-2.66966966966967,0)
--(axis cs:-2.67567567567568,0)
--(axis cs:-2.68168168168168,0)
--(axis cs:-2.68768768768769,0)
--(axis cs:-2.69369369369369,0)
--(axis cs:-2.6996996996997,0)
--(axis cs:-2.70570570570571,0)
--(axis cs:-2.71171171171171,0)
--(axis cs:-2.71771771771772,0)
--(axis cs:-2.72372372372372,0)
--(axis cs:-2.72972972972973,0)
--(axis cs:-2.73573573573574,0)
--(axis cs:-2.74174174174174,0)
--(axis cs:-2.74774774774775,0)
--(axis cs:-2.75375375375375,0)
--(axis cs:-2.75975975975976,0)
--(axis cs:-2.76576576576577,0)
--(axis cs:-2.77177177177177,0)
--(axis cs:-2.77777777777778,0)
--(axis cs:-2.78378378378378,0)
--(axis cs:-2.78978978978979,0)
--(axis cs:-2.7957957957958,0)
--(axis cs:-2.8018018018018,0)
--(axis cs:-2.80780780780781,0)
--(axis cs:-2.81381381381381,0)
--(axis cs:-2.81981981981982,0)
--(axis cs:-2.82582582582583,0)
--(axis cs:-2.83183183183183,0)
--(axis cs:-2.83783783783784,0)
--(axis cs:-2.84384384384384,0)
--(axis cs:-2.84984984984985,0)
--(axis cs:-2.85585585585586,0)
--(axis cs:-2.86186186186186,0)
--(axis cs:-2.86786786786787,0)
--(axis cs:-2.87387387387387,0)
--(axis cs:-2.87987987987988,0)
--(axis cs:-2.88588588588589,0)
--(axis cs:-2.89189189189189,0)
--(axis cs:-2.8978978978979,0)
--(axis cs:-2.9039039039039,0)
--(axis cs:-2.90990990990991,0)
--(axis cs:-2.91591591591592,0)
--(axis cs:-2.92192192192192,0)
--(axis cs:-2.92792792792793,0)
--(axis cs:-2.93393393393393,0)
--(axis cs:-2.93993993993994,0)
--(axis cs:-2.94594594594595,0)
--(axis cs:-2.95195195195195,0)
--(axis cs:-2.95795795795796,0)
--(axis cs:-2.96396396396396,0)
--(axis cs:-2.96996996996997,0)
--(axis cs:-2.97597597597598,0)
--(axis cs:-2.98198198198198,0)
--(axis cs:-2.98798798798799,0)
--(axis cs:-2.99399399399399,0)
--(axis cs:-3,0)
--cycle;

\addplot [semithick, steelblue31119180]
table {%
-3 0.00443184841193801
-2.99399399399399 0.0045123438886223
-2.98798798798799 0.00459413567812293
-2.98198198198198 0.00467724132352912
-2.97597597597598 0.00476167854213413
-2.96996996996997 0.00484746522600703
-2.96396396396396 0.00493461944254309
-2.95795795795796 0.0050231594349922
-2.95195195195195 0.00511310362296472
-2.94594594594595 0.00520447060291445
-2.93993993993994 0.0052972791485979
-2.93393393393393 0.0053915482115097
-2.92792792792793 0.00548729692129329
-2.92192192192192 0.00558454458612664
-2.91591591591592 0.00568331069308223
-2.90990990990991 0.00578361490846092
-2.9039039039039 0.00588547707809912
-2.8978978978979 0.00598891722764874
-2.89189189189189 0.00609395556282928
-2.88588588588589 0.00620061246965171
-2.87987987987988 0.00630890851461339
-2.87387387387387 0.00641886444486359
-2.86786786786787 0.00653050118833907
-2.86186186186186 0.0066438398538691
-2.85585585585586 0.00675890173124943
-2.84984984984985 0.0068757082912846
-2.84384384384384 0.00699428118579803
-2.83783783783784 0.00711464224760944
-2.83183183183183 0.00723681349047888
-2.82582582582583 0.00736081710901693
-2.81981981981982 0.00748667547856039
-2.81381381381381 0.00761441115501307
-2.80780780780781 0.00774404687465088
-2.8018018018018 0.00787560555389093
-2.7957957957958 0.0080091102890238
-2.78978978978979 0.00814458435590864
-2.78378378378378 0.00828205120963033
-2.77777777777778 0.00842153448411835
-2.77177177177177 0.00856305799172663
-2.76576576576577 0.00870664572277382
-2.75975975975976 0.00885232184504357
-2.75375375375375 0.00900011070324404
-2.74774774774775 0.00915003681842624
-2.74174174174174 0.00930212488736056
-2.73573573573574 0.00945639978187097
-2.72972972972973 0.00961288654812628
-2.72372372372372 0.00977161040588793
-2.71771771771772 0.00993259674771377
-2.71171171171171 0.0100958711381172
-2.70570570570571 0.0102614593126814
-2.6996996996997 0.0104293871771275
-2.69369369369369 0.0105996808063366
-2.68768768768769 0.0107723664433257
-2.68168168168168 0.0109474704981749
-2.67567567567568 0.0111250195469083
-2.66966966966967 0.0113050403303254
-2.66366366366366 0.011487559752784
-2.65765765765766 0.0116726048809339
-2.65165165165165 0.0118602029423997
-2.64564564564565 0.0120503813244144
-2.63963963963964 0.0122431675724008
-2.63363363363363 0.0124385893885022
-2.62762762762763 0.0126366746300606
-2.62162162162162 0.0128374513080428
-2.61561561561562 0.0130409475854126
-2.60960960960961 0.0132471917754508
-2.6036036036036 0.0134562123400202
-2.5975975975976 0.013668037887776
-2.59159159159159 0.0138826971723226
-2.58558558558559 0.0141002190903132
-2.57957957957958 0.0143206326794947
-2.57357357357357 0.0145439671166962
-2.56756756756757 0.0147702517157603
-2.56156156156156 0.0149995159254171
-2.55555555555556 0.0152317893271017
-2.54954954954955 0.0154671016327119
-2.54354354354354 0.015705482682309
-2.53753753753754 0.015946962441759
-2.53153153153153 0.0161915710003145
-2.52552552552553 0.016439338568137
-2.51951951951952 0.0166902954737598
-2.51351351351351 0.0169444721614894
-2.50750750750751 0.0172018991887471
-2.5015015015015 0.0174626072233491
-2.4954954954955 0.017726627040725
-2.48948948948949 0.0179939895210751
-2.48348348348348 0.0182647256464652
-2.47747747747748 0.0185388664978589
-2.47147147147147 0.0188164432520877
-2.46546546546547 0.0190974871787576
-2.45945945945946 0.0193820296370931
-2.45345345345345 0.0196701020727171
-2.44744744744745 0.0199617360143675
-2.44144144144144 0.0202569630705496
-2.43543543543544 0.0205558149261244
-2.42942942942943 0.0208583233388324
-2.42342342342342 0.0211645201357533
-2.41741741741742 0.0214744372097002
-2.41141141141141 0.021788106515549
-2.40540540540541 0.0221055600665036
-2.3993993993994 0.0224268299302945
-2.39339339339339 0.0227519482253133
-2.38738738738739 0.023080947116681
-2.38138138138138 0.0234138588122505
-2.37537537537538 0.0237507155585446
-2.36936936936937 0.0240915496366271
-2.36336336336336 0.0244363933579092
-2.35735735735736 0.0247852790598887
-2.35135135135135 0.025138239101825
-2.34534534534535 0.0254953058603477
-2.33933933933934 0.0258565117249987
-2.33333333333333 0.0262218890937095
-2.32732732732733 0.0265914703682128
-2.32132132132132 0.0269652879493876
-2.31531531531532 0.0273433742325399
-2.30930930930931 0.0277257616026172
-2.3033033033033 0.0281124824293578
-2.2972972972973 0.0285035690623752
-2.29129129129129 0.0288990538261777
-2.28528528528529 0.0292989690151221
-2.27927927927928 0.0297033468883044
-2.27327327327327 0.0301122196643845
-2.26726726726727 0.0305256195163479
-2.26126126126126 0.030943578566203
-2.25525525525526 0.0313661288796147
-2.24924924924925 0.0317933024604746
-2.24324324324324 0.0322251312454088
-2.23723723723724 0.0326616470982222
-2.23123123123123 0.0331028818042809
-2.22522522522523 0.0335488670648327
-2.21921921921922 0.0339996344912656
-2.21321321321321 0.0344552155993059
-2.20720720720721 0.0349156418031548
-2.2012012012012 0.0353809444095654
-2.1951951951952 0.0358511546118594
-2.18918918918919 0.0363263034838852
-2.18318318318318 0.0368064219739166
-2.17717717717718 0.0372915408984941
-2.17117117117117 0.0377816909362074
-2.16516516516517 0.0382769026214225
-2.15915915915916 0.0387772063379509
-2.15315315315315 0.0392826323126641
-2.14714714714715 0.0397932106090522
-2.14114114114114 0.0403089711207292
-2.13513513513514 0.040829943564883
-2.12912912912913 0.041356157475674
-2.12312312312312 0.0418876421975804
-2.11711711711712 0.0424244268786927
-2.11111111111111 0.0429665404639572
-2.10510510510511 0.0435140116883696
-2.0990990990991 0.04406686907012
-2.09309309309309 0.0446251409036892
-2.08708708708709 0.0451888552528987
-2.08108108108108 0.0457580399439133
-2.07507507507508 0.0463327225581989
-2.06906906906907 0.0469129304254359
-2.06306306306306 0.0474986906163891
-2.05705705705706 0.0480900299357348
-2.05105105105105 0.0486869749148476
-2.04504504504505 0.0492895518045451
-2.03903903903904 0.0498977865677951
-2.03303303303303 0.0505117048723834
-2.02702702702703 0.0511313320835454
-2.02102102102102 0.0517566932565611
-2.01501501501502 0.0523878131293159
-2.00900900900901 0.0530247161148276
-2.003003003003 0.0536674262937412
-1.996996996997 0.0543159674067926
-1.99099099099099 0.0549703628472423
-1.98498498498498 0.0556306356532813
-1.97897897897898 0.0562968085004086
-1.97297297297297 0.0569689036937837
-1.96696696696697 0.0576469431605548
-1.96096096096096 0.0583309484421626
-1.95495495495495 0.0590209406866238
-1.94894894894895 0.0597169406407933
-1.94294294294294 0.0604189686426077
-1.93693693693694 0.061127044613312
-1.93093093093093 0.0618411880496693
-1.92492492492492 0.0625614180161572
-1.91891891891892 0.0632877531371501
-1.91291291291291 0.0640202115890914
-1.90690690690691 0.0647588110926551
-1.9009009009009 0.0655035689048992
-1.89489489489489 0.0662545018114133
-1.88888888888889 0.0670116261184602
-1.88288288288288 0.0677749576451152
-1.87687687687688 0.0685445117154032
-1.87087087087087 0.0693203031504359
-1.86486486486486 0.070102346260551
-1.85885885885886 0.0708906548374551
-1.85285285285285 0.0716852421463716
-1.84684684684685 0.0724861209181953
-1.84084084084084 0.0732933033416567
-1.83483483483483 0.0741068010554964
-1.82882882882883 0.0749266251406517
-1.82282282282282 0.0757527861124582
-1.81681681681682 0.0765852939128673
-1.81081081081081 0.077424157902681
-1.8048048048048 0.0782693868538078
-1.7987987987988 0.07912098894154
-1.79279279279279 0.0799789717368538
-1.78678678678679 0.0808433421987368
-1.78078078078078 0.0817141066665415
-1.77477477477477 0.0825912708523695
-1.76876876876877 0.0834748398334868
-1.76276276276276 0.0843648180447727
-1.75675675675676 0.0852612092712038
-1.75075075075075 0.0861640166403769
-1.74474474474474 0.0870732426150701
-1.73873873873874 0.0879888889858462
-1.73273273273273 0.0889109568636995
-1.72672672672673 0.0898394466727489
-1.72072072072072 0.0907743581429781
-1.71471471471471 0.0917156903030263
-1.70870870870871 0.0926634414730305
-1.7027027027027 0.0936176092575221
-1.6966966966967 0.0945781905383797
-1.69069069069069 0.0955451814678399
-1.68468468468468 0.0965185774615691
-1.67867867867868 0.0974983731917974
-1.67267267267267 0.0984845625805171
-1.66666666666667 0.0994771387927487
-1.66066066066066 0.100476094229875
-1.65465465465465 0.101481420523047
-1.64864864864865 0.102493108526664
-1.64264264264264 0.103511148311924
-1.63663663663664 0.10453552916046
-1.63063063063063 0.105566239558052
-1.62462462462462 0.106603267188415
-1.61861861861862 0.107646598927082
-1.61261261261261 0.108696220835366
-1.60660660660661 0.109752118154414
-1.6006006006006 0.110814275299345
-1.59459459459459 0.111882675853491
-1.58858858858859 0.112957302562724
-1.58258258258258 0.114038137329885
-1.57657657657658 0.115125161209309
-1.57057057057057 0.116218354401455
-1.56456456456456 0.117317696247634
-1.55855855855856 0.118423165224848
-1.55255255255255 0.119534738940736
-1.54654654654655 0.120652394128624
-1.54054054054054 0.121776106642695
-1.53453453453453 0.122905851453265
-1.52852852852853 0.124041602642185
-1.52252252252252 0.125183333398351
-1.51651651651652 0.126331016013341
-1.51051051051051 0.12748462187717
-1.5045045045045 0.128644121474174
-1.4984984984985 0.129809484379015
-1.49249249249249 0.13098067925282
-1.48648648648649 0.132157673839447
-1.48048048048048 0.133340434961882
-1.47447447447447 0.134528928518776
-1.46846846846847 0.135723119481111
-1.46246246246246 0.136922971889012
-1.45645645645646 0.138128448848694
-1.45045045045045 0.139339512529552
-1.44444444444444 0.140556124161395
-1.43843843843844 0.141778244031828
-1.43243243243243 0.143005831483783
-1.42642642642643 0.14423884491319
-1.42042042042042 0.145477241766812
-1.41441441441441 0.146720978540224
-1.40840840840841 0.147970010775955
-1.4024024024024 0.149224293061777
-1.3963963963964 0.150483779029159
-1.39039039039039 0.151748421351882
-1.38438438438438 0.153018171744808
-1.37837837837838 0.154292980962825
-1.37237237237237 0.155572798799941
-1.36636636636637 0.15685757408856
-1.36036036036036 0.158147254698914
-1.35435435435435 0.159441787538676
-1.34834834834835 0.160741118552729
-1.34234234234234 0.162045192723125
-1.33633633633634 0.163353954069202
-1.33033033033033 0.164667345647889
-1.32432432432432 0.165985309554181
-1.31831831831832 0.167307786921791
-1.31231231231231 0.168634717923986
-1.30630630630631 0.169966041774598
-1.3003003003003 0.171301696729227
-1.29429429429429 0.172641620086611
-1.28828828828829 0.173985748190197
-1.28228228228228 0.175334016429883
-1.27627627627628 0.176686359243956
-1.27027027027027 0.178042710121214
-1.26426426426426 0.179403001603276
-1.25825825825826 0.180767165287079
-1.25225225225225 0.182135131827571
-1.24624624624625 0.183506830940589
-1.24024024024024 0.184882191405935
-1.23423423423423 0.186261141070638
-1.22822822822823 0.187643606852412
-1.22222222222222 0.189029514743312
-1.21621621621622 0.190418789813581
-1.21021021021021 0.191811356215688
-1.2042042042042 0.193207137188574
-1.1981981981982 0.194606055062081
-1.19219219219219 0.196008031261586
-1.18618618618619 0.197412986312834
-1.18018018018018 0.19882083984696
-1.17417417417417 0.200231510605714
-1.16816816816817 0.20164491644689
-1.16216216216216 0.203060974349942
-1.15615615615616 0.204479600421811
-1.15015015015015 0.205900709902938
-1.14414414414414 0.207324217173486
-1.13813813813814 0.20875003575976
-1.13213213213213 0.210178078340819
-1.12612612612613 0.211608256755296
-1.12012012012012 0.213040482008409
-1.11411411411411 0.214474664279177
-1.10810810810811 0.215910712927831
-1.1021021021021 0.217348536503421
-1.0960960960961 0.218788042751629
-1.09009009009009 0.22022913862277
-1.08408408408408 0.221671730279998
-1.07807807807808 0.2231157231077
-1.07207207207207 0.224561021720095
-1.06606606606607 0.226007529970025
-1.06006006006006 0.227455150957937
-1.05405405405405 0.228903787041065
-1.04804804804805 0.230353339842804
-1.04204204204204 0.231803710262273
-1.03603603603604 0.233254798484074
-1.03003003003003 0.23470650398824
-1.02402402402402 0.236158725560373
-1.01801801801802 0.237611361301974
-1.01201201201201 0.239064308640953
-1.00600600600601 0.240517464342334
-1 0.241970724519143
-0.993993993993994 0.243423984643478
-0.987987987987988 0.244877139557762
-0.981981981981982 0.246330083486184
-0.975975975975976 0.247782710046314
-0.96996996996997 0.249234912260894
-0.963963963963964 0.250686582569819
-0.957957957957958 0.252137612842278
-0.951951951951952 0.253587894389084
-0.945945945945946 0.255037317975163
-0.93993993993994 0.256485773832225
-0.933933933933934 0.257933151671597
-0.927927927927928 0.259379340697227
-0.921921921921922 0.260824229618852
-0.915915915915916 0.262267706665329
-0.90990990990991 0.263709659598126
-0.903903903903904 0.265149975724979
-0.897897897897898 0.266588541913699
-0.891891891891892 0.268025244606136
-0.885885885885886 0.269459969832298
-0.87987987987988 0.270892603224623
-0.873873873873874 0.272323030032391
-0.867867867867868 0.273751135136292
-0.861861861861862 0.275176803063131
-0.855855855855856 0.276599918000677
-0.84984984984985 0.278020363812654
-0.843843843843844 0.279438024053859
-0.837837837837838 0.280852781985427
-0.831831831831832 0.282264520590213
-0.825825825825826 0.283673122588316
-0.81981981981982 0.285078470452718
-0.813813813813814 0.286480446425052
-0.807807807807808 0.287878932531491
-0.801801801801802 0.289273810598746
-0.795795795795796 0.29066496227019
-0.78978978978979 0.292052269022087
-0.783783783783784 0.293435612179928
-0.777777777777778 0.294814872934877
-0.771771771771772 0.29618993236032
-0.765765765765766 0.297560671428505
-0.75975975975976 0.298926971027289
-0.753753753753754 0.300288711976973
-0.747747747747748 0.301645775047225
-0.741741741741742 0.302998040974092
-0.735735735735736 0.304345390477095
-0.72972972972973 0.305687704276405
-0.723723723723724 0.307024863110094
-0.717717717717718 0.308356747751458
-0.711711711711712 0.309683239026412
-0.705705705705706 0.311004217830952
-0.6996996996997 0.312319565148675
-0.693693693693694 0.313629162068366
-0.687687687687688 0.314932889801632
-0.681681681681682 0.316230629700595
-0.675675675675676 0.317522263275628
-0.66966966966967 0.318807672213141
-0.663663663663664 0.320086738393407
-0.657657657657658 0.321359343908417
-0.651651651651652 0.322625371079786
-0.645645645645646 0.32388470247667
-0.63963963963964 0.325137220933723
-0.633633633633634 0.326382809569071
-0.627627627627628 0.327621351802301
-0.621621621621621 0.328852731372475
-0.615615615615616 0.330076832356145
-0.60960960960961 0.331293539185378
-0.603603603603604 0.332502736665788
-0.597597597597598 0.333704309994563
-0.591591591591591 0.334898144778487
-0.585585585585586 0.336084127051958
-0.57957957957958 0.337262143294989
-0.573573573573574 0.338432080451194
-0.567567567567568 0.339593825945755
-0.561561561561561 0.340747267703363
-0.555555555555555 0.341892294166129
-0.54954954954955 0.34302879431147
-0.543543543543544 0.344156657669946
-0.537537537537538 0.345275774343073
-0.531531531531531 0.346386035021075
-0.525525525525525 0.3474873310006
-0.51951951951952 0.34857955420238
-0.513513513513514 0.349662597188831
-0.507507507507508 0.350736353181597
-0.501501501501501 0.351800716079031
-0.495495495495495 0.352855580473602
-0.48948948948949 0.353900841669234
-0.483483483483484 0.35493639569857
-0.477477477477477 0.355962139340152
-0.471471471471471 0.356977970135516
-0.465465465465465 0.357983786406208
-0.45945945945946 0.358979487270697
-0.453453453453454 0.359964972661194
-0.447447447447447 0.360940143340381
-0.441441441441441 0.361904900918023
-0.435435435435435 0.36285914786748
-0.42942942942943 0.363802787542104
-0.423423423423424 0.364735724191528
-0.417417417417417 0.365657862977822
-0.411411411411411 0.36656910999154
-0.405405405405405 0.36746937226763
-0.3993993993994 0.368358557801224
-0.393393393393394 0.369236575563278
-0.387387387387387 0.370103335516092
-0.381381381381381 0.370958748628674
-0.375375375375375 0.371802726891965
-0.36936936936937 0.372635183333916
-0.363363363363364 0.373456032034409
-0.357357357357357 0.374265188140023
-0.351351351351351 0.375062567878638
-0.345345345345345 0.375848088573884
-0.33933933933934 0.376621668659409
-0.333333333333333 0.377383227692993
-0.327327327327327 0.378132686370474
-0.321321321321321 0.378869966539505
-0.315315315315315 0.379594991213131
-0.30930930930931 0.380307684583177
-0.303303303303303 0.381007972033453
-0.297297297297297 0.381695780152771
-0.291291291291291 0.382371036747761
-0.285285285285285 0.383033670855497
-0.279279279279279 0.383683612755922
-0.273273273273273 0.384320793984073
-0.267267267267267 0.38494514734209
-0.261261261261261 0.385556606911032
-0.255255255255255 0.386155108062468
-0.249249249249249 0.386740587469861
-0.243243243243243 0.387312983119735
-0.237237237237237 0.387872234322616
-0.231231231231231 0.388418281723758
-0.225225225225225 0.388951067313638
-0.219219219219219 0.389470534438229
-0.213213213213213 0.389976627809035
-0.207207207207207 0.390469293512899
-0.201201201201201 0.390948479021577
-0.195195195195195 0.391414133201066
-0.189189189189189 0.391866206320701
-0.183183183183183 0.392304650062005
-0.177177177177177 0.392729417527296
-0.171171171171171 0.393140463248047
-0.165165165165165 0.393537743192998
-0.159159159159159 0.393921214776013
-0.153153153153153 0.394290836863694
-0.147147147147147 0.39464656978273
-0.141141141141141 0.394988375326995
-0.135135135135135 0.395316216764386
-0.129129129129129 0.395630058843405
-0.123123123123123 0.395929867799471
-0.117117117117117 0.39621561136098
-0.111111111111111 0.396487258755088
-0.105105105105105 0.396744780713241
-0.099099099099099 0.396988149476425
-0.0930930930930933 0.397217338800158
-0.0870870870870872 0.397432323959201
-0.0810810810810811 0.39763308175201
-0.075075075075075 0.397819590504904
-0.069069069069069 0.397991830075968
-0.0630630630630633 0.398149781858677
-0.0570570570570572 0.398293428785244
-0.0510510510510511 0.3984227553297
-0.045045045045045 0.398537747510682
-0.0390390390390389 0.39863839289396
-0.0330330330330328 0.398724680594675
-0.0270270270270272 0.398796601279297
-0.0210210210210211 0.398854147167316
-0.015015015015015 0.398897312032637
-0.00900900900900892 0.398926091204706
-0.00300300300300282 0.398940481569353
0.00300300300300282 0.398940481569353
0.00900900900900892 0.398926091204706
0.015015015015015 0.398897312032637
0.0210210210210211 0.398854147167316
0.0270270270270272 0.398796601279297
0.0330330330330328 0.398724680594675
0.0390390390390389 0.39863839289396
0.045045045045045 0.398537747510682
0.0510510510510511 0.3984227553297
0.0570570570570572 0.398293428785244
0.0630630630630629 0.398149781858677
0.069069069069069 0.397991830075968
0.075075075075075 0.397819590504904
0.0810810810810811 0.39763308175201
0.0870870870870872 0.397432323959201
0.0930930930930929 0.397217338800158
0.099099099099099 0.396988149476425
0.105105105105105 0.396744780713241
0.111111111111111 0.396487258755088
0.117117117117117 0.39621561136098
0.123123123123123 0.395929867799471
0.129129129129129 0.395630058843405
0.135135135135135 0.395316216764386
0.141141141141141 0.394988375326995
0.147147147147147 0.39464656978273
0.153153153153153 0.394290836863694
0.159159159159159 0.393921214776013
0.165165165165165 0.393537743192998
0.171171171171171 0.393140463248047
0.177177177177177 0.392729417527296
0.183183183183183 0.392304650062005
0.189189189189189 0.391866206320701
0.195195195195195 0.391414133201066
0.201201201201201 0.390948479021577
0.207207207207207 0.390469293512899
0.213213213213213 0.389976627809035
0.219219219219219 0.389470534438229
0.225225225225225 0.388951067313638
0.231231231231231 0.388418281723758
0.237237237237237 0.387872234322616
0.243243243243243 0.387312983119735
0.249249249249249 0.386740587469861
0.255255255255255 0.386155108062468
0.261261261261261 0.385556606911032
0.267267267267267 0.38494514734209
0.273273273273273 0.384320793984073
0.279279279279279 0.383683612755922
0.285285285285285 0.383033670855497
0.291291291291291 0.382371036747761
0.297297297297297 0.381695780152771
0.303303303303303 0.381007972033453
0.309309309309309 0.380307684583177
0.315315315315315 0.379594991213131
0.321321321321321 0.378869966539505
0.327327327327327 0.378132686370474
0.333333333333333 0.377383227692993
0.339339339339339 0.376621668659409
0.345345345345345 0.375848088573884
0.351351351351351 0.375062567878638
0.357357357357357 0.374265188140023
0.363363363363364 0.373456032034409
0.369369369369369 0.372635183333916
0.375375375375375 0.371802726891965
0.381381381381381 0.370958748628674
0.387387387387387 0.370103335516092
0.393393393393394 0.369236575563278
0.399399399399399 0.368358557801224
0.405405405405405 0.36746937226763
0.411411411411411 0.36656910999154
0.417417417417417 0.365657862977822
0.423423423423424 0.364735724191528
0.429429429429429 0.363802787542104
0.435435435435435 0.36285914786748
0.441441441441441 0.361904900918023
0.447447447447447 0.360940143340381
0.453453453453454 0.359964972661194
0.45945945945946 0.358979487270697
0.465465465465465 0.357983786406208
0.471471471471471 0.356977970135516
0.477477477477477 0.355962139340152
0.483483483483484 0.35493639569857
0.48948948948949 0.353900841669234
0.495495495495495 0.352855580473602
0.501501501501501 0.351800716079031
0.507507507507508 0.350736353181597
0.513513513513514 0.349662597188831
0.51951951951952 0.34857955420238
0.525525525525525 0.3474873310006
0.531531531531531 0.346386035021075
0.537537537537538 0.345275774343073
0.543543543543544 0.344156657669946
0.54954954954955 0.34302879431147
0.555555555555555 0.341892294166129
0.561561561561561 0.340747267703363
0.567567567567568 0.339593825945755
0.573573573573574 0.338432080451194
0.57957957957958 0.337262143294989
0.585585585585585 0.336084127051958
0.591591591591591 0.334898144778487
0.597597597597598 0.333704309994563
0.603603603603604 0.332502736665788
0.60960960960961 0.331293539185378
0.615615615615615 0.330076832356145
0.621621621621621 0.328852731372475
0.627627627627628 0.327621351802301
0.633633633633634 0.326382809569071
0.63963963963964 0.325137220933723
0.645645645645645 0.32388470247667
0.651651651651652 0.322625371079786
0.657657657657658 0.321359343908417
0.663663663663664 0.320086738393407
0.66966966966967 0.318807672213141
0.675675675675675 0.317522263275628
0.681681681681682 0.316230629700595
0.687687687687688 0.314932889801632
0.693693693693694 0.313629162068366
0.6996996996997 0.312319565148675
0.705705705705705 0.311004217830952
0.711711711711712 0.309683239026412
0.717717717717718 0.308356747751458
0.723723723723724 0.307024863110094
0.72972972972973 0.305687704276405
0.735735735735736 0.304345390477095
0.741741741741742 0.302998040974092
0.747747747747748 0.301645775047225
0.753753753753754 0.300288711976973
0.75975975975976 0.298926971027289
0.765765765765766 0.297560671428505
0.771771771771772 0.29618993236032
0.777777777777778 0.294814872934877
0.783783783783784 0.293435612179928
0.78978978978979 0.292052269022087
0.795795795795796 0.29066496227019
0.801801801801802 0.289273810598746
0.807807807807808 0.287878932531491
0.813813813813814 0.286480446425052
0.81981981981982 0.285078470452718
0.825825825825826 0.283673122588316
0.831831831831832 0.282264520590213
0.837837837837838 0.280852781985427
0.843843843843844 0.279438024053859
0.84984984984985 0.278020363812654
0.855855855855856 0.276599918000677
0.861861861861862 0.275176803063131
0.867867867867868 0.273751135136292
0.873873873873874 0.272323030032391
0.87987987987988 0.270892603224623
0.885885885885886 0.269459969832298
0.891891891891892 0.268025244606136
0.897897897897898 0.266588541913699
0.903903903903904 0.265149975724979
0.90990990990991 0.263709659598126
0.915915915915916 0.262267706665329
0.921921921921922 0.260824229618852
0.927927927927928 0.259379340697227
0.933933933933934 0.257933151671597
0.93993993993994 0.256485773832225
0.945945945945946 0.255037317975163
0.951951951951952 0.253587894389084
0.957957957957958 0.252137612842278
0.963963963963964 0.250686582569819
0.96996996996997 0.249234912260894
0.975975975975976 0.247782710046314
0.981981981981982 0.246330083486184
0.987987987987988 0.244877139557762
0.993993993993994 0.243423984643478
1 0.241970724519143
1.00600600600601 0.240517464342334
1.01201201201201 0.239064308640953
1.01801801801802 0.237611361301974
1.02402402402402 0.236158725560373
1.03003003003003 0.23470650398824
1.03603603603604 0.233254798484074
1.04204204204204 0.231803710262273
1.04804804804805 0.230353339842804
1.05405405405405 0.228903787041065
1.06006006006006 0.227455150957937
1.06606606606607 0.226007529970025
1.07207207207207 0.224561021720095
1.07807807807808 0.2231157231077
1.08408408408408 0.221671730279998
1.09009009009009 0.22022913862277
1.0960960960961 0.218788042751629
1.1021021021021 0.217348536503421
1.10810810810811 0.215910712927831
1.11411411411411 0.214474664279177
1.12012012012012 0.213040482008409
1.12612612612613 0.211608256755296
1.13213213213213 0.210178078340819
1.13813813813814 0.20875003575976
1.14414414414414 0.207324217173486
1.15015015015015 0.205900709902938
1.15615615615616 0.204479600421811
1.16216216216216 0.203060974349942
1.16816816816817 0.20164491644689
1.17417417417417 0.200231510605714
1.18018018018018 0.19882083984696
1.18618618618619 0.197412986312835
1.19219219219219 0.196008031261586
1.1981981981982 0.194606055062081
1.2042042042042 0.193207137188574
1.21021021021021 0.191811356215688
1.21621621621622 0.190418789813581
1.22222222222222 0.189029514743312
1.22822822822823 0.187643606852412
1.23423423423423 0.186261141070638
1.24024024024024 0.184882191405935
1.24624624624625 0.183506830940589
1.25225225225225 0.182135131827571
1.25825825825826 0.180767165287079
1.26426426426426 0.179403001603276
1.27027027027027 0.178042710121214
1.27627627627628 0.176686359243956
1.28228228228228 0.175334016429883
1.28828828828829 0.173985748190197
1.29429429429429 0.172641620086611
1.3003003003003 0.171301696729227
1.30630630630631 0.169966041774598
1.31231231231231 0.168634717923986
1.31831831831832 0.167307786921791
1.32432432432432 0.165985309554181
1.33033033033033 0.164667345647889
1.33633633633634 0.163353954069202
1.34234234234234 0.162045192723125
1.34834834834835 0.160741118552729
1.35435435435435 0.159441787538676
1.36036036036036 0.158147254698914
1.36636636636637 0.156857574088559
1.37237237237237 0.155572798799941
1.37837837837838 0.154292980962825
1.38438438438438 0.153018171744808
1.39039039039039 0.151748421351882
1.3963963963964 0.150483779029159
1.4024024024024 0.149224293061777
1.40840840840841 0.147970010775955
1.41441441441441 0.146720978540224
1.42042042042042 0.145477241766812
1.42642642642643 0.14423884491319
1.43243243243243 0.143005831483783
1.43843843843844 0.141778244031829
1.44444444444444 0.140556124161394
1.45045045045045 0.139339512529552
1.45645645645646 0.138128448848694
1.46246246246246 0.136922971889012
1.46846846846847 0.135723119481111
1.47447447447447 0.134528928518776
1.48048048048048 0.133340434961882
1.48648648648649 0.132157673839447
1.49249249249249 0.13098067925282
1.4984984984985 0.129809484379015
1.5045045045045 0.128644121474174
1.51051051051051 0.12748462187717
1.51651651651652 0.126331016013341
1.52252252252252 0.125183333398351
1.52852852852853 0.124041602642185
1.53453453453453 0.122905851453265
1.54054054054054 0.121776106642695
1.54654654654655 0.120652394128624
1.55255255255255 0.119534738940736
1.55855855855856 0.118423165224849
1.56456456456456 0.117317696247634
1.57057057057057 0.116218354401455
1.57657657657658 0.115125161209309
1.58258258258258 0.114038137329885
1.58858858858859 0.112957302562724
1.59459459459459 0.111882675853491
1.6006006006006 0.110814275299345
1.60660660660661 0.109752118154414
1.61261261261261 0.108696220835366
1.61861861861862 0.107646598927082
1.62462462462462 0.106603267188415
1.63063063063063 0.105566239558052
1.63663663663664 0.10453552916046
1.64264264264264 0.103511148311924
1.64864864864865 0.102493108526664
1.65465465465465 0.101481420523047
1.66066066066066 0.100476094229875
1.66666666666667 0.0994771387927487
1.67267267267267 0.0984845625805171
1.67867867867868 0.0974983731917975
1.68468468468468 0.0965185774615691
1.69069069069069 0.09554518146784
1.6966966966967 0.0945781905383796
1.7027027027027 0.0936176092575221
1.70870870870871 0.0926634414730305
1.71471471471471 0.0917156903030263
1.72072072072072 0.0907743581429781
1.72672672672673 0.0898394466727489
1.73273273273273 0.0889109568636995
1.73873873873874 0.0879888889858463
1.74474474474474 0.0870732426150701
1.75075075075075 0.086164016640377
1.75675675675676 0.0852612092712037
1.76276276276276 0.0843648180447727
1.76876876876877 0.0834748398334869
1.77477477477477 0.0825912708523695
1.78078078078078 0.0817141066665415
1.78678678678679 0.0808433421987367
1.79279279279279 0.0799789717368538
1.7987987987988 0.07912098894154
1.8048048048048 0.0782693868538078
1.81081081081081 0.077424157902681
1.81681681681682 0.0765852939128672
1.82282282282282 0.0757527861124582
1.82882882882883 0.0749266251406517
1.83483483483483 0.0741068010554964
1.84084084084084 0.0732933033416568
1.84684684684685 0.0724861209181953
1.85285285285285 0.0716852421463716
1.85885885885886 0.0708906548374552
1.86486486486486 0.070102346260551
1.87087087087087 0.0693203031504359
1.87687687687688 0.0685445117154032
1.88288288288288 0.0677749576451153
1.88888888888889 0.0670116261184602
1.89489489489489 0.0662545018114133
1.9009009009009 0.0655035689048993
1.90690690690691 0.064758811092655
1.91291291291291 0.0640202115890914
1.91891891891892 0.06328775313715
1.92492492492492 0.0625614180161572
1.93093093093093 0.0618411880496694
1.93693693693694 0.061127044613312
1.94294294294294 0.0604189686426077
1.94894894894895 0.0597169406407932
1.95495495495495 0.0590209406866238
1.96096096096096 0.0583309484421627
1.96696696696697 0.0576469431605548
1.97297297297297 0.0569689036937837
1.97897897897898 0.0562968085004085
1.98498498498498 0.0556306356532813
1.99099099099099 0.0549703628472424
1.996996996997 0.0543159674067925
2.003003003003 0.0536674262937412
2.00900900900901 0.0530247161148276
2.01501501501502 0.0523878131293159
2.02102102102102 0.0517566932565612
2.02702702702703 0.0511313320835454
2.03303303303303 0.0505117048723834
2.03903903903904 0.0498977865677951
2.04504504504505 0.0492895518045451
2.05105105105105 0.0486869749148476
2.05705705705706 0.0480900299357348
2.06306306306306 0.0474986906163891
2.06906906906907 0.0469129304254359
2.07507507507508 0.0463327225581989
2.08108108108108 0.0457580399439134
2.08708708708709 0.0451888552528987
2.09309309309309 0.0446251409036892
2.0990990990991 0.0440668690701199
2.10510510510511 0.0435140116883696
2.11111111111111 0.0429665404639572
2.11711711711712 0.0424244268786927
2.12312312312312 0.0418876421975804
2.12912912912913 0.0413561574756739
2.13513513513514 0.040829943564883
2.14114114114114 0.0403089711207292
2.14714714714715 0.0397932106090522
2.15315315315315 0.0392826323126641
2.15915915915916 0.0387772063379509
2.16516516516517 0.0382769026214225
2.17117117117117 0.0377816909362075
2.17717717717718 0.0372915408984941
2.18318318318318 0.0368064219739166
2.18918918918919 0.0363263034838851
2.1951951951952 0.0358511546118594
2.2012012012012 0.0353809444095654
2.20720720720721 0.0349156418031548
2.21321321321321 0.0344552155993059
2.21921921921922 0.0339996344912656
2.22522522522523 0.0335488670648327
2.23123123123123 0.033102881804281
2.23723723723724 0.0326616470982222
2.24324324324324 0.0322251312454088
2.24924924924925 0.0317933024604746
2.25525525525526 0.0313661288796147
2.26126126126126 0.0309435785662031
2.26726726726727 0.0305256195163479
2.27327327327327 0.0301122196643845
2.27927927927928 0.0297033468883044
2.28528528528529 0.0292989690151221
2.29129129129129 0.0288990538261777
2.2972972972973 0.0285035690623752
2.3033033033033 0.0281124824293578
2.30930930930931 0.0277257616026172
2.31531531531532 0.0273433742325399
2.32132132132132 0.0269652879493876
2.32732732732733 0.0265914703682128
2.33333333333333 0.0262218890937095
2.33933933933934 0.0258565117249987
2.34534534534535 0.0254953058603477
2.35135135135135 0.0251382391018251
2.35735735735736 0.0247852790598887
2.36336336336336 0.0244363933579092
2.36936936936937 0.0240915496366271
2.37537537537538 0.0237507155585446
2.38138138138138 0.0234138588122505
2.38738738738739 0.023080947116681
2.39339339339339 0.0227519482253134
2.3993993993994 0.0224268299302945
2.40540540540541 0.0221055600665036
2.41141141141141 0.021788106515549
2.41741741741742 0.0214744372097002
2.42342342342342 0.0211645201357534
2.42942942942943 0.0208583233388324
2.43543543543544 0.0205558149261244
2.44144144144144 0.0202569630705496
2.44744744744745 0.0199617360143675
2.45345345345345 0.0196701020727171
2.45945945945946 0.0193820296370931
2.46546546546547 0.0190974871787576
2.47147147147147 0.0188164432520877
2.47747747747748 0.0185388664978589
2.48348348348348 0.0182647256464652
2.48948948948949 0.0179939895210751
2.4954954954955 0.017726627040725
2.5015015015015 0.017462607223349
2.50750750750751 0.0172018991887471
2.51351351351351 0.0169444721614894
2.51951951951952 0.0166902954737598
2.52552552552553 0.016439338568137
2.53153153153153 0.0161915710003145
2.53753753753754 0.015946962441759
2.54354354354354 0.015705482682309
2.54954954954955 0.0154671016327119
2.55555555555556 0.0152317893271017
2.56156156156156 0.0149995159254171
2.56756756756757 0.0147702517157603
2.57357357357357 0.0145439671166962
2.57957957957958 0.0143206326794947
2.58558558558559 0.0141002190903132
2.59159159159159 0.0138826971723226
2.5975975975976 0.013668037887776
2.6036036036036 0.0134562123400202
2.60960960960961 0.0132471917754508
2.61561561561562 0.0130409475854126
2.62162162162162 0.0128374513080428
2.62762762762763 0.0126366746300606
2.63363363363363 0.0124385893885022
2.63963963963964 0.0122431675724008
2.64564564564565 0.0120503813244144
2.65165165165165 0.0118602029423997
2.65765765765766 0.0116726048809339
2.66366366366366 0.0114875597527841
2.66966966966967 0.0113050403303254
2.67567567567568 0.0111250195469083
2.68168168168168 0.0109474704981749
2.68768768768769 0.0107723664433257
2.69369369369369 0.0105996808063367
2.6996996996997 0.0104293871771275
2.70570570570571 0.0102614593126814
2.71171171171171 0.0100958711381172
2.71771771771772 0.00993259674771377
2.72372372372372 0.00977161040588794
2.72972972972973 0.00961288654812628
2.73573573573574 0.00945639978187098
2.74174174174174 0.00930212488736055
2.74774774774775 0.00915003681842624
2.75375375375375 0.00900011070324405
2.75975975975976 0.00885232184504357
2.76576576576577 0.00870664572277383
2.77177177177177 0.00856305799172662
2.77777777777778 0.00842153448411835
2.78378378378378 0.00828205120963034
2.78978978978979 0.00814458435590864
2.7957957957958 0.00800911028902381
2.8018018018018 0.00787560555389092
2.80780780780781 0.00774404687465088
2.81381381381381 0.00761441115501308
2.81981981981982 0.00748667547856039
2.82582582582583 0.00736081710901694
2.83183183183183 0.00723681349047888
2.83783783783784 0.00711464224760944
2.84384384384384 0.00699428118579803
2.84984984984985 0.0068757082912846
2.85585585585586 0.00675890173124944
2.86186186186186 0.0066438398538691
2.86786786786787 0.00653050118833907
2.87387387387387 0.00641886444486359
2.87987987987988 0.00630890851461339
2.88588588588589 0.00620061246965171
2.89189189189189 0.00609395556282928
2.8978978978979 0.00598891722764874
2.9039039039039 0.00588547707809912
2.90990990990991 0.00578361490846092
2.91591591591592 0.00568331069308224
2.92192192192192 0.00558454458612664
2.92792792792793 0.00548729692129329
2.93393393393393 0.00539154821150969
2.93993993993994 0.0052972791485979
2.94594594594595 0.00520447060291445
2.95195195195195 0.00511310362296472
2.95795795795796 0.0050231594349922
2.96396396396396 0.00493461944254308
2.96996996996997 0.00484746522600703
2.97597597597598 0.00476167854213414
2.98198198198198 0.00467724132352912
2.98798798798799 0.00459413567812293
2.99399399399399 0.0045123438886223
3 0.00443184841193801
};
\addplot [semithick, steelblue31119180]
table {%
-4 0.000133830225764885
-3.9959979989995 0.000135988730829575
-3.991995997999 0.000138179836626972
-3.9879939969985 0.000140403997682941
-3.983991995998 0.000142661674242141
-3.9799899949975 0.000144953332330358
-3.975987993997 0.000147279443817371
-3.9719859929965 0.000149640486480351
-3.967983991996 0.000152036944067809
-3.9639819909955 0.000154469306364083
-3.959979989995 0.000156938069254369
-3.9559779889945 0.000159443734790312
-3.951975987994 0.000161986811256128
-3.9479739869935 0.0001645678132353
-3.943971985993 0.000167187261677817
-3.9399699849925 0.000169845683967974
-3.935967983992 0.000172543613992735
-3.9319659829915 0.000175281592210656
-3.927963981991 0.000178060165721374
-3.9239619809905 0.000180879888335659
-3.91995997998999 0.000183741320646047
-3.91595797898949 0.00018664503009803
-3.91195597798899 0.000189591591061832
-3.90795397698849 0.000192581584904751
-3.90395197598799 0.000195615600064085
-3.89994997498749 0.000198694232120639
-3.89594797398699 0.000201818083872808
-3.89194597298649 0.000204987765411252
-3.88794397198599 0.000208203894194149
-3.88394197098549 0.000211467095123042
-3.87993996998499 0.00021477800061927
-3.87593796898449 0.000218137250701002
-3.87193596798399 0.000221545493060847
-3.86793396698349 0.000225003383144077
-3.86393196598299 0.000228511584227435
-3.85992996498249 0.000232070767498547
-3.85592796398199 0.000235681612135933
-3.85192596298149 0.00023934480538962
-3.84792396198099 0.000243061042662359
-3.84392196098049 0.000246831027591447
-3.83991995997999 0.000250655472131154
-3.83591795897949 0.000254535096635762
-3.83191595797899 0.000258470629943209
-3.82791395697849 0.000262462809459342
-3.82391195597799 0.000266512381242796
-3.81990995497749 0.000270620100090461
-3.81590795397699 0.000274786729623591
-3.81190595297649 0.00027901304237451
-3.80790395197599 0.000283299819873939
-3.80390195097549 0.000287647852738946
-3.79989994997499 0.000292057940761511
-3.79589794897449 0.000296530892997706
-3.79189594797399 0.000301067527857504
-3.78789394697349 0.000305668673195204
-3.78389194597299 0.000310335166400482
-3.77988994497249 0.000315067854490059
-3.77588794397199 0.000319867594199995
-3.77188594297149 0.000324735252078613
-3.76788394197099 0.000329671704580038
-3.76388194097049 0.000334677838158367
-3.75987993996999 0.000339754549362459
-3.75587793896948 0.000344902744931364
-3.75187593796898 0.000350123341890356
-3.74787393696848 0.000355417267647614
-3.74387193596798 0.000360785460091515
-3.73986993496748 0.000366228867688561
-3.73586793396698 0.000371748449581928
-3.73186593296648 0.000377345175690644
-3.72786393196598 0.000383020026809389
-3.72386193096548 0.000388773994708932
-3.71985992996498 0.000394608082237173
-3.71585792896448 0.000400523303420834
-3.71185592796398 0.000406520683567759
-3.70785392696348 0.000412601259369842
-3.70385192596298 0.000418766079006579
-3.69984992496248 0.000425016202249248
-3.69584792396198 0.0004313527005657
-3.69184592296148 0.000437776657225783
-3.68784392196098 0.000444289167407379
-3.68384192096048 0.000450891338303061
-3.67983991995998 0.000457584289227376
-3.67583791895948 0.000464369151724731
-3.67183591795898 0.000471247069677907
-3.66783391695848 0.000478219199417186
-3.66383191595798 0.00048528670983008
-3.65982991495748 0.00049245078247169
-3.65582791395698 0.000499712611675657
-3.65182591295648 0.000507073404665731
-3.64782391195598 0.000514534381667943
-3.64382191095548 0.000522096776023387
-3.63981990995498 0.000529761834301592
-3.63581790895448 0.000537530816414505
-3.63181590795398 0.000545404995731075
-3.62781390695348 0.000553385659192413
-3.62381190595298 0.000561474107427576
-3.61980990495248 0.000569671654869907
-3.61580790395198 0.000577979629873993
-3.61180590295148 0.000586399374833193
-3.60780390195098 0.00059493224629775
-3.60380190095048 0.000603579615093484
-3.59979989994997 0.000612342866441066
-3.59579789894947 0.000621223400075859
-3.59179589794897 0.00063022263036833
-3.58779389694847 0.000639341986445035
-3.58379189594797 0.000648582912310156
-3.57978989494747 0.000657946866967615
-3.57578789394697 0.000667435324543723
-3.57178589294647 0.000677049774410397
-3.56778389194597 0.000686791721308922
-3.56378189094547 0.000696662685474251
-3.55977988994497 0.00070666420275985
-3.55577788894447 0.000716797824763082
-3.55177588794397 0.000727065118951111
-3.54777388694347 0.000737467668787344
-3.54377188594297 0.000748007073858395
-3.53976988494247 0.00075868495000155
-3.53576788394197 0.000769502929432767
-3.53176588294147 0.00078046266087517
-3.52776388194097 0.000791565809688039
-3.52376188094047 0.000802814057996319
-3.51975987993997 0.000814209104820593
-3.51575787893947 0.000825752666207558
-3.51175587793897 0.000837446475360981
-3.50775387693847 0.000849292282773117
-3.50375187593797 0.00086129185635661
-3.49974987493747 0.00087344698157684
-3.49574787393697 0.000885759461584736
-3.49174587293647 0.000898231117350046
-3.48774387193597 0.000910863787795024
-3.48374187093547 0.000923659329928578
-3.47973986993497 0.000936619618980837
-3.47573786893447 0.000949746548538137
-3.47173586793397 0.000963042030678428
-3.46773386693347 0.00097650799610709
-3.46373186593297 0.000990146394293136
-3.45972986493247 0.00100395919360582
-3.45572786393197 0.00101794838145165
-3.45172586293147 0.0010321159644117
-3.44772386193097 0.00104646396837941
-3.44372186093047 0.00106099443869865
-3.43971985992996 0.00107570944030215
-3.43571785892946 0.00109061105785036
-3.43171585792896 0.00110570139587049
-3.42771385692846 0.00112098257889603
-3.42371185592796 0.00113645675160653
-3.41970985492746 0.0011521260789676
-3.41570785392696 0.0011679927463714
-3.41170585292646 0.00118405895977723
-3.40770385192596 0.00120032694585249
-3.40370185092546 0.00121679895211396
-3.39969984992496 0.0012334772470692
-3.39569784892446 0.00125036412035834
-3.39169584792396 0.00126746188289601
-3.38769384692346 0.00128477286701359
-3.38369184592296 0.00130229942660157
-3.37968984492246 0.00132004393725222
-3.37568784392196 0.00133800879640242
-3.37168584292146 0.0013561964234766
-3.36768384192096 0.00137460926003005
-3.36368184092046 0.00139324976989214
-3.35967983991996 0.0014121204393099
-3.35567783891946 0.00143122377709165
-3.35167583791896 0.00145056231475072
-3.34767383691846 0.0014701386066494
-3.34367183591796 0.00148995523014288
-3.33966983491746 0.00151001478572335
-3.33566783391696 0.00153031989716415
-3.33166583291646 0.00155087321166397
-3.32766383191596 0.00157167739999117
-3.32366183091546 0.00159273515662804
-3.31965982991496 0.00161404919991516
-3.31565782891446 0.00163562227219575
-3.31165582791396 0.00165745713996003
-3.30765382691346 0.00167955659398951
-3.30365182591296 0.00170192344950131
-3.29964982491246 0.00172456054629244
-3.29564782391196 0.00174747074888396
-3.29164582291146 0.00177065694666511
-3.28764382191096 0.00179412205403736
-3.28364182091046 0.00181786901055832
-3.27963981990996 0.00184190078108557
-3.27563781890945 0.0018662203559203
-3.27163581790895 0.00189083075095087
-3.26763381690845 0.00191573500779617
-3.26363181590795 0.00194093619394875
-3.25962981490745 0.00196643740291785
-3.25562781390695 0.00199224175437215
-3.25162581290645 0.00201835239428227
-3.24762381190595 0.00204477249506308
-3.24362181090545 0.00207150525571571
-3.23961980990495 0.00209855390196929
-3.23561780890445 0.00212592168642238
-3.23161580790395 0.00215361188868406
-3.22761380690345 0.00218162781551477
-3.22361180590295 0.00220997280096666
-3.21960980490245 0.0022386502065237
-3.21560780390195 0.00226766342124134
-3.21160580290145 0.00229701586188575
-3.20760380190095 0.00232671097307262
-3.20360180090045 0.00235675222740563
-3.19959979989995 0.00238714312561427
-3.19559779889945 0.00241788719669131
-3.19159579789895 0.00244898799802976
-3.18759379689845 0.00248044911555922
-3.18359179589795 0.00251227416388174
-3.17958979489745 0.00254446678640717
-3.17558779389695 0.0025770306554878
-3.17158579289645 0.00260996947255252
-3.16758379189595 0.00264328696824025
-3.16358179089545 0.00267698690253281
-3.15957978989495 0.00271107306488704
-3.15557778889445 0.00274554927436635
-3.15157578789395 0.00278041937977138
-3.14757378689345 0.0028156872597701
-3.14357178589295 0.00285135682302709
-3.13956978489245 0.00288743200833198
-3.13556778389195 0.00292391678472722
-3.13156578289145 0.0029608151516349
-3.12756378189095 0.00299813113898285
-3.12356178089045 0.00303586880732974
-3.11955977988995 0.00307403224798938
-3.11555777888944 0.00311262558315415
-3.11155577788894 0.00315165296601731
-3.10755377688844 0.00319111858089458
-3.10355177588794 0.00323102664334457
-3.09954977488744 0.00327138140028826
-3.09554777388694 0.00331218713012745
-3.09154577288644 0.00335344814286218
-3.08754377188594 0.00339516878020699
-3.08354177088544 0.00343735341570615
-3.07953976988494 0.00348000645484772
-3.07553776888444 0.00352313233517649
-3.07153576788394 0.00356673552640568
-3.06753376688344 0.00361082053052751
-3.06353176588294 0.00365539188192245
-3.05952976488244 0.00370045414746732
-3.05552776388194 0.00374601192664197
-3.05152576288144 0.00379206985163481
-3.04752376188094 0.00383863258744684
-3.04352176088044 0.00388570483199446
-3.03951975987994 0.00393329131621078
-3.03551775887944 0.0039813968041456
-3.03151575787894 0.00403002609306386
-3.02751375687844 0.00407918401354273
-3.02351175587794 0.00412887542956707
-3.01950975487744 0.00417910523862349
-3.01550775387694 0.00422987837179276
-3.01150575287644 0.0042811997938407
-3.00750375187594 0.00433307450330743
-3.00350175087544 0.00438550753259501
-2.99949974987494 0.00443850394805337
-2.99549774887444 0.00449206885006458
-2.99149574787394 0.00454620737312541
-2.98749374687344 0.00460092468592809
-2.98349174587294 0.00465622599143934
-2.97948974487244 0.00471211652697757
-2.97548774387194 0.00476860156428819
-2.97148574287144 0.00482568640961718
-2.96748374187094 0.00488337640378263
-2.96348174087044 0.00494167692224441
-2.95947973986994 0.0050005933751719
-2.95547773886944 0.00506013120750968
-2.95147573786893 0.00512029589904129
-2.94747373686843 0.00518109296445079
-2.94347173586793 0.00524252795338246
-2.93946973486743 0.00530460645049818
-2.93546773386693 0.0053673340755328
-2.93146573286643 0.00543071648334727
-2.92746373186593 0.00549475936397966
-2.92346173086543 0.00555946844269383
-2.91945972986493 0.00562484948002593
-2.91545772886443 0.00569090827182858
-2.91145572786393 0.00575765064931271
-2.90745372686343 0.00582508247908701
-2.90345172586293 0.00589320966319512
-2.89944972486243 0.00596203813915028
-2.89544772386193 0.00603157387996753
-2.89144572286143 0.00610182289419349
-2.88744372186093 0.00617279122593363
-2.88344172086043 0.0062444849548769
-2.87943971985993 0.00631691019631789
-2.87543771885943 0.00639007310117632
-2.87143571785893 0.00646397985601387
-2.86743371685843 0.00653863668304842
-2.86343171585793 0.00661404984016549
-2.85942971485743 0.00669022562092694
-2.85542771385693 0.00676717035457702
-2.85142571285643 0.00684489040604547
-2.84742371185593 0.00692339217594783
-2.84342171085543 0.00700268210058292
-2.83941970985493 0.00708276665192742
-2.83541770885443 0.00716365233762738
-2.83141570785393 0.00724534570098696
-2.82741370685343 0.00732785332095404
-2.82341170585293 0.00741118181210278
-2.81940970485243 0.00749533782461324
-2.81540770385193 0.00758032804424776
-2.81140570285143 0.00766615919232429
-2.80740370185093 0.0077528380256865
-2.80340170085043 0.0078403713366708
-2.79939969984992 0.00792876595306993
-2.79539769884942 0.00801802873809349
-2.79139569784892 0.00810816659032506
-2.78739369684842 0.00819918644367601
-2.78339169584792 0.00829109526733593
-2.77938969484742 0.00838390006571971
-2.77538769384692 0.00847760787841116
-2.77138569284642 0.00857222578010313
-2.76738369184592 0.0086677608805342
-2.76338169084542 0.00876422032442184
-2.75937968984492 0.00886161129139188
-2.75537768884442 0.00895994099590457
-2.75137568784392 0.00905921668717695
-2.74737368684342 0.00915944564910138
-2.74337168584292 0.00926063520016071
-2.73936968484242 0.00936279269333944
-2.73536768384192 0.00946592551603126
-2.73136568284142 0.00957004108994271
-2.72736368184092 0.00967514687099317
-2.72336168084042 0.00978125034921077
-2.71935967983992 0.00988835904862461
-2.71535767883942 0.00999648052715299
-2.71135567783892 0.0101056223764876
-2.70735367683842 0.0102157922219739
-2.70335167583792 0.0103269977224873
-2.69934967483742 0.0104392465703055
-2.69534767383692 0.0105525464909763
-2.69134567283642 0.0106669052431821
-2.68734367183592 0.0107823306185994
-2.68334167083542 0.0108988304417546
-2.67933966983492 0.0110164125698756
-2.67533766883442 0.011135084892739
-2.67133566783392 0.011254855332513
-2.66733366683342 0.0113757318435959
-2.66333166583292 0.0114977224124512
-2.65932966483242 0.0116208350574366
-2.65532766383192 0.0117450778286302
-2.65132566283142 0.0118704588076518
-2.64732366183092 0.011996986107479
-2.64332166083042 0.0121246678722601
-2.63931965982992 0.0122535122771213
-2.63531765882941 0.0123835275279703
-2.63131565782891 0.0125147218612947
-2.62731365682841 0.0126471035439559
-2.62331165582791 0.0127806808729789
-2.61930965482741 0.0129154621753364
-2.61530765382691 0.0130514558077297
-2.61130565282641 0.0131886701563631
-2.60730365182591 0.013327113636715
-2.60330165082541 0.0134667946933038
-2.59929964982491 0.0136077217994486
-2.59529764882441 0.0137499034570256
-2.59129564782391 0.0138933481962193
-2.58729364682341 0.0140380645752692
-2.58329164582291 0.0141840611802108
-2.57928964482241 0.0143313466246131
-2.57528764382191 0.0144799295493096
-2.57128564282141 0.0146298186221248
-2.56728364182091 0.0147810225375969
-2.56328164082041 0.0149335500166932
-2.55927963981991 0.0150874098065224
-2.55527763881941 0.0152426106800412
-2.55127563781891 0.0153991614357553
-2.54727363681841 0.0155570708974155
-2.54327163581791 0.0157163479137096
-2.53926963481741 0.0158770013579475
-2.53526763381691 0.0160390401277425
-2.53126563281641 0.0162024731446864
-2.52726363181591 0.0163673093540201
-2.52326163081541 0.0165335577242984
-2.51925962981491 0.0167012272470496
-2.51525762881441 0.0168703269364303
-2.51125562781391 0.0170408658288742
-2.50725362681341 0.0172128529827356
-2.50325162581291 0.0173862974779283
-2.49924962481241 0.0175612084155583
-2.49524762381191 0.017737594917551
-2.49124562281141 0.0179154661262741
-2.48724362181091 0.0180948312041537
-2.48324162081041 0.0182756993332856
-2.47923961980991 0.0184580797150412
-2.4752376188094 0.018641981569668
-2.4712356178089 0.0188274141358837
-2.4672336168084 0.0190143866704659
-2.4632316158079 0.0192029084478357
-2.4592296148074 0.0193929887596357
-2.4552276138069 0.0195846369143029
-2.4512256128064 0.0197778622366349
-2.4472236118059 0.0199726740673522
-2.4432216108054 0.0201690817626532
-2.4392196098049 0.0203670946937649
-2.4352176088044 0.0205667222464867
-2.4312156078039 0.02076797382073
-2.4272136068034 0.0209708588300509
-2.4232116058029 0.0211753867011778
-2.4192096048024 0.0213815668735334
-2.4152076038019 0.0215894087987504
-2.4112056028014 0.0217989219401825
-2.4072036018009 0.0220101157724088
-2.4032016008004 0.022222999780733
-2.3991995997999 0.0224375834606764
-2.3951975987994 0.0226538763174659
-2.3911955977989 0.0228718878655154
-2.3871935967984 0.0230916276279024
-2.3831915957979 0.0233131051358379
-2.3791895947974 0.0235363299281311
-2.3751875937969 0.0237613115506488
-2.3711855927964 0.0239880595557675
-2.3671835917959 0.0242165835018218
-2.3631815907954 0.0244468929525453
-2.3591795897949 0.0246789974765068
-2.3551775887944 0.02491290664654
-2.3511755877939 0.0251486300391684
-2.3471735867934 0.0253861772340233
-2.3431715857929 0.0256255578132569
-2.3391695847924 0.0258667813609492
-2.3351675837919 0.0261098574625096
-2.3311655827914 0.0263547957040721
-2.3271635817909 0.0266016056718852
-2.3231615807904 0.0268502969516961
-2.3191595797899 0.0271008791281289
-2.31515757878939 0.0273533617840575
-2.31115557778889 0.0276077544999719
-2.30715357678839 0.0278640668533401
-2.30315157578789 0.0281223084179631
-2.29914957478739 0.0283824887633247
-2.29514757378689 0.0286446174539358
-2.29114557278639 0.0289087040486728
-2.28714357178589 0.0291747581001104
-2.28314157078539 0.0294427891538482
-2.27913956978489 0.0297128067478332
-2.27513756878439 0.0299848204116746
-2.27113556778389 0.0302588396659547
-2.26713356678339 0.0305348740215332
-2.26313156578289 0.0308129329788468
-2.25912956478239 0.0310930260272019
-2.25512756378189 0.0313751626440633
-2.25112556278139 0.0316593522943363
-2.24712356178089 0.0319456044296436
-2.24312156078039 0.0322339284875968
-2.23911955977989 0.0325243338910628
-2.23511755877939 0.0328168300474235
-2.23111555777889 0.0331114263478317
-2.22711355677839 0.0334081321664603
-2.22311155577789 0.033706956859747
-2.21910955477739 0.0340079097656331
-2.21510755377689 0.0343110002027972
-2.21110555277639 0.0346162374698836
-2.20710355177589 0.0349236308447251
-2.20310155077539 0.0352331895835617
-2.19909954977489 0.0355449229202521
-2.19509754877439 0.0358588400654819
-2.19109554777389 0.0361749502059658
-2.18709354677339 0.0364932625036447
-2.18309154577289 0.0368137860948774
-2.17908954477239 0.0371365300896285
-2.17508754377189 0.0374615035706496
-2.17108554277139 0.0377887155926566
-2.16708354177089 0.0381181751815018
-2.16308154077039 0.038449891333341
-2.15907953976988 0.0387838730137959
-2.15507753876939 0.039120129157111
-2.15107553776888 0.0394586686653071
-2.14707353676838 0.0397995004073282
-2.14307153576788 0.0401426332181854
-2.13906953476738 0.040488075898095
-2.13506753376688 0.0408358372116122
-2.13106553276638 0.0411859258867605
-2.12706353176588 0.0415383506141568
-2.12306153076538 0.0418931200461305
-2.11905952976488 0.0422502427958404
-2.11505752876438 0.0426097274363852
-2.11105552776388 0.0429715824999109
-2.10705352676338 0.043335816476713
-2.10305152576288 0.0437024378143353
-2.09904952476238 0.0440714549166642
-2.09504752376188 0.0444428761430178
-2.09104552276138 0.0448167098072328
-2.08704352176088 0.045192964176746
-2.08304152076038 0.0455716474716717
-2.07903951975988 0.0459527678638764
-2.07503751875938 0.0463363334760481
-2.07103551775888 0.046722352380763
-2.06703351675838 0.0471108325995476
-2.06303151575788 0.0475017821019373
-2.05902951475738 0.047895208804532
-2.05502751375688 0.0482911205700467
-2.05102551275638 0.0486895252063603
-2.04702351175588 0.0490904304655595
-2.04302151075538 0.0494938440429799
-2.03901950975488 0.0498997735762438
-2.03501750875438 0.0503082266442947
-2.03101550775388 0.0507192107664284
-2.02701350675338 0.0511327334013212
-2.02301150575288 0.0515488019460546
-2.01900950475238 0.0519674237351377
-2.01500750375188 0.0523886060395261
-2.01100550275138 0.0528123560656379
-2.00700350175088 0.0532386809543676
-2.00300150075038 0.0536675877800961
-1.99899949974988 0.0540990835496996
-1.99499749874937 0.0545331752015543
-1.99099549774887 0.0549698696045397
-1.98699349674837 0.0554091735570391
-1.98299149574787 0.055851093785938
-1.97898949474737 0.0562956369456199
-1.97498749374687 0.0567428096169597
-1.97098549274637 0.0571926183063163
-1.96698349174587 0.057645069444521
-1.96298149074537 0.0581001693858663
-1.95897948974487 0.0585579244070909
-1.95497748874437 0.0590183407063641
-1.95097548774387 0.0594814244022683
-1.94697348674337 0.0599471815327793
-1.94297148574287 0.0604156180542458
-1.93896948474237 0.0608867398403671
-1.93496748374187 0.0613605526811695
-1.93096548274137 0.0618370622819813
-1.92696348174087 0.0623162742624073
-1.92296148074037 0.0627981941553008
-1.91895947973987 0.0632828274057365
-1.91495747873937 0.0637701793699806
-1.91095547773887 0.0642602553144615
-1.90695347673837 0.0647530604147393
-1.90295147573787 0.0652485997544744
-1.89894947473737 0.0657468783243956
-1.89494747373687 0.0662479010212688
-1.89094547273637 0.0667516726468635
-1.88694347173587 0.067258197906921
-1.88294147073537 0.067767481410121
-1.87893946973487 0.0682795276670489
-1.87493746873437 0.068794341089163
-1.87093546773387 0.0693119259877618
-1.86693346673337 0.0698322865729514
-1.86293146573287 0.070355426952614
-1.85892946473237 0.0708813511313756
-1.85492746373187 0.0714100630095753
-1.85092546273137 0.071941566382235
-1.84692346173087 0.0724758649380299
-1.84292146073037 0.0730129622582592
-1.83891945972987 0.0735528618158191
-1.83491745872936 0.0740955669741756
-1.83091545772886 0.0746410809863395
-1.82691345672836 0.0751894069938418
-1.82291145572786 0.0757405480257113
-1.81890945472736 0.0762945069974531
-1.81490745372686 0.0768512867100293
-1.81090545272636 0.077410889848841
-1.80690345172586 0.0779733189827123
-1.80290145072536 0.0785385765628764
-1.79889944972486 0.0791066649219632
-1.79489744872436 0.0796775862729903
-1.79089544772386 0.0802513427083548
-1.78689344672336 0.0808279361988287
-1.78289144572286 0.0814073685925561
-1.77888944472236 0.0819896416140534
-1.77488744372186 0.0825747568632125
-1.77088544272136 0.0831627158143061
-1.76688344172086 0.0837535198149964
-1.76288144072036 0.0843471700853476
-1.75887943971986 0.0849436677168402
-1.75487743871936 0.08554301367139
-1.75087543771886 0.0861452087803697
-1.74687343671836 0.0867502537436344
-1.74287143571786 0.087358149128551
-1.73886943471736 0.0879688953690308
-1.73486743371686 0.0885824927645666
-1.73086543271636 0.0891989414792741
-1.72686343171586 0.0898182415409364
-1.72286143071536 0.0904403928400536
-1.71885942971486 0.0910653951288975
-1.71485742871436 0.0916932480205689
-1.71085542771386 0.0923239509880614
-1.70685342671336 0.0929575033633293
-1.70285142571286 0.0935939043363602
-1.69884942471236 0.0942331529542528
-1.69484742371186 0.0948752481203003
-1.69084542271136 0.0955201885930785
-1.68684342171086 0.0961679729855392
-1.68284142071036 0.09681859976411
-1.67883941970986 0.0974720672477988
-1.67483741870935 0.0981283736073049
-1.67083541770885 0.0987875168641351
-1.66683341670835 0.0994494948897264
-1.66283141570785 0.100114305404575
-1.65882941470735 0.100781945977373
-1.65482741370685 0.101452414024144
-1.65082541270635 0.1021257068074
-1.64682341170585 0.102801821435289
-1.64282141070535 0.103480754860759
-1.63881940970485 0.104162503880726
-1.63481740870435 0.10484706513525
-1.63081540770385 0.105534435106718
-1.62681340670335 0.10622461011903
-1.62281140570285 0.106917586336804
-1.61880940470235 0.107613359764571
-1.61480740370185 0.108311926245997
-1.61080540270135 0.109013281463097
-1.60680340170085 0.109717420935463
-1.60280140070035 0.110424340019505
-1.59879939969985 0.111134033907692
-1.59479739869935 0.111846497627806
-1.59079539769885 0.1125617260422
-1.58679339669835 0.113279713847074
-1.58279139569785 0.114000455571745
-1.57878939469735 0.114723945577943
-1.57478739369685 0.1154501780591
-1.57078539269635 0.116179147039658
-1.56678339169585 0.116910846374384
-1.56278139069535 0.117645269747692
-1.55877938969485 0.118382410672978
-1.55477738869435 0.119122262491959
-1.55077538769385 0.119864818374028
-1.54677338669335 0.120610071315618
-1.54277138569285 0.121358014139567
-1.53876938469235 0.122108639494507
-1.53476738369185 0.122861939854253
-1.53076538269135 0.123617907517205
-1.52676338169085 0.124376534605762
-1.52276138069035 0.125137813065745
-1.51875937968985 0.125901734665829
-1.51475737868934 0.126668290996991
-1.51075537768884 0.127437473471964
-1.50675337668834 0.128209273324702
-1.50275137568784 0.12898368160986
-1.49874937468734 0.129760689202283
-1.49474737368684 0.130540286796504
-1.49074537268634 0.131322464906257
-1.48674337168584 0.132107213864001
-1.48274137068534 0.132894523820454
-1.47873936968484 0.133684384744142
-1.47473736868434 0.134476786420952
-1.47073536768384 0.135271718453711
-1.46673336668334 0.136069170261763
-1.46273136568284 0.136869131080565
-1.45872936468234 0.137671589961297
-1.45472736368184 0.13847653577048
-1.45072536268134 0.13928395718961
-1.44672336168084 0.140093842714802
-1.44272136068034 0.14090618065645
-1.43871935967984 0.141720959138896
-1.43471735867934 0.142538166100115
-1.43071535767884 0.143357789291415
-1.42671335667834 0.144179816277144
-1.42271135567784 0.145004234434413
-1.41870935467734 0.145831030952839
-1.41470735367684 0.14666019283429
-1.41070535267634 0.147491706892653
-1.40670335167584 0.148325559753611
-1.40270135067534 0.149161737854435
-1.39869934967484 0.150000227443793
-1.39469734867434 0.150841014581564
-1.39069534767384 0.151684085138679
-1.38669334667334 0.152529424796967
-1.38269134567284 0.153377019049015
-1.37868934467234 0.15422685319805
-1.37468734367184 0.155078912357829
-1.37068534267134 0.155933181452544
-1.36668334167084 0.156789645216746
-1.36268134067034 0.157648288195279
-1.35867933966983 0.158509094743231
-1.35467733866933 0.1593720490259
-1.35067533766883 0.160237135018777
-1.34667333666833 0.161104336507539
-1.34267133566783 0.161973637088059
-1.33866933466733 0.16284502016644
-1.33466733366683 0.163718468959047
-1.33066533266633 0.164593966492573
-1.32666333166583 0.165471495604105
-1.32266133066533 0.166351038941218
-1.31865932966483 0.167232578962077
-1.31465732866433 0.168116097935554
-1.31065532766383 0.16900157794137
-1.30665332666333 0.169889000870243
-1.30265132566283 0.170778348424055
-1.29864932466233 0.171669602116037
-1.29464732366183 0.17256274327097
-1.29064532266133 0.173457753025399
-1.28664332166083 0.174354612327867
-1.28264132066033 0.175253301939161
-1.27863931965983 0.17615380243258
-1.27463731865933 0.177056094194214
-1.27063531765883 0.177960157423245
-1.26663331665833 0.178865972132256
-1.26263131565783 0.179773518147568
-1.25862931465733 0.180682775109583
-1.25462731365683 0.181593722473152
-1.25062531265633 0.18250633950795
-1.24662331165583 0.183420605298883
-1.24262131065533 0.184336498746495
-1.23861930965483 0.1852539985674
-1.23461730865433 0.186173083294738
-1.23061530765383 0.187093731278629
-1.22661330665333 0.188015920686666
-1.22261130565283 0.188939629504407
-1.21860930465233 0.189864835535896
-1.21460730365183 0.190791516404196
-1.21060530265133 0.191719649551935
-1.20660330165083 0.192649212241882
-1.20260130065033 0.193580181557527
-1.19859929964983 0.194512534403684
-1.19459729864932 0.195446247507111
-1.19059529764882 0.196381297417147
-1.18659329664832 0.197317660506369
-1.18259129564782 0.198255312971257
-1.17858929464732 0.199194230832891
-1.17458729364682 0.200134389937651
-1.17058529264632 0.201075765957943
-1.16658329164582 0.202018334392941
-1.16258129064532 0.202962070569344
-1.15857928964482 0.203906949642153
-1.15457728864432 0.204852946595462
-1.15057528764382 0.205800036243273
-1.14657328664332 0.206748193230319
-1.14257128564282 0.207697392032913
-1.13856928464232 0.20864760695981
-1.13456728364182 0.209598812153087
-1.13056528264132 0.210550981589042
-1.12656328164082 0.211504089079106
-1.12256128064032 0.212458108270781
-1.11855927963982 0.213413012648585
-1.11455727863932 0.214368775535021
-1.11055527763882 0.215325370091563
-1.10655327663832 0.216282769319656
-1.10255127563782 0.217240946061737
-1.09854927463732 0.21819987300227
-1.09454727363682 0.219159522668802
-1.09054527263632 0.220119867433033
-1.08654327163582 0.221080879511904
-1.08254127063532 0.222042530968703
-1.07853926963482 0.22300479371419
-1.07453726863432 0.223967639507732
-1.07053526763382 0.224931039958466
-1.06653326663332 0.225894966526467
-1.06253126563282 0.226859390523945
-1.05852926463232 0.227824283116448
-1.05452726363182 0.228789615324091
-1.05052526263132 0.229755358022795
-1.04652326163082 0.230721481945548
-1.04252126063032 0.231687957683678
-1.03851925962981 0.23265475568815
-1.03451725862931 0.233621846270869
-1.03051525762881 0.234589199606009
-1.02651325662831 0.235556785731354
-1.02251125562781 0.23652457454966
-1.01850925462731 0.237492535830026
-1.01450725362681 0.238460639209285
-1.01050525262631 0.239428854193418
-1.00650325162581 0.240397150158971
-1.00250125062531 0.241365496354498
-0.998499249624813 0.242333861902022
-0.994497248624312 0.243302215798498
-0.990495247623812 0.244270526917309
-0.986493246623312 0.245238764009768
-0.982491245622811 0.246206895706638
-0.978489244622311 0.247174890519666
-0.974487243621811 0.248142716843138
-0.970485242621311 0.249110342955442
-0.966483241620811 0.250077737020655
-0.96248124062031 0.251044867090135
-0.95847923961981 0.252011701104139
-0.95447723861931 0.252978206893449
-0.950475237618809 0.253944352181015
-0.94647323661831 0.254910104583613
-0.942471235617809 0.25587543161352
-0.938469234617309 0.2568403006802
-0.934467233616809 0.257804679092006
-0.930465232616308 0.2587685340579
-0.926463231615808 0.259731832689179
-0.922461230615308 0.260694542001228
-0.918459229614808 0.261656628915273
-0.914457228614308 0.26261806026016
-0.910455227613807 0.263578802774141
-0.906453226613307 0.264538823106676
-0.902451225612807 0.265498087820248
-0.898449224612306 0.266456563392195
-0.894447223611806 0.267414216216547
-0.890445222611306 0.26837101260589
-0.886443221610806 0.269326918793228
-0.882441220610305 0.270281900933869
-0.878439219609805 0.271235925107322
-0.874437218609305 0.272188957319199
-0.870435217608805 0.273140963503143
-0.866433216608304 0.274091909522756
-0.862431215607804 0.275041761173547
-0.858429214607304 0.275990484184891
-0.854427213606804 0.276938044221996
-0.850425212606303 0.277884406887888
-0.846423211605803 0.278829537725403
-0.842421210605303 0.279773402219196
-0.838419209604802 0.280715965797751
-0.834417208604302 0.281657193835416
-0.830415207603802 0.282597051654442
-0.826413206603302 0.283535504527028
-0.822411205602802 0.28447251767739
-0.818409204602301 0.285408056283828
-0.814407203601801 0.286342085480809
-0.810405202601301 0.287274570361064
-0.8064032016008 0.288205475977687
-0.802401200600301 0.289134767346251
-0.7983991995998 0.290062409446931
-0.7943971985993 0.290988367226634
-0.7903951975988 0.291912605601146
-0.786393196598299 0.292835089457281
-0.782391195597799 0.293755783655043
-0.778389194597299 0.294674653029796
-0.774387193596799 0.295591662394442
-0.770385192596299 0.29650677654161
-0.766383191595798 0.297419960245848
-0.762381190595298 0.298331178265834
-0.758379189594798 0.299240395346582
-0.754377188594297 0.300147576221666
-0.750375187593797 0.301052685615446
-0.746373186593297 0.301955688245302
-0.742371185592797 0.30285654882388
-0.738369184592296 0.303755232061342
-0.734367183591796 0.304651702667618
-0.730365182591296 0.305545925354673
-0.726363181590796 0.306437864838778
-0.722361180590295 0.307327485842785
-0.718359179589795 0.308214753098406
-0.714357178589295 0.309099631348508
-0.710355177588795 0.309982085349404
-0.706353176588294 0.310862079873149
-0.702351175587794 0.311739579709853
-0.698349174587294 0.312614549669984
-0.694347173586793 0.313486954586685
-0.690345172586293 0.314356759318096
-0.686343171585793 0.315223928749676
-0.682341170585293 0.316088427796532
-0.678339169584793 0.316950221405752
-0.674337168584292 0.31780927455874
-0.670335167583792 0.318665552273557
-0.666333166583292 0.319519019607265
-0.662331165582791 0.320369641658273
-0.658329164582292 0.321217383568686
-0.654327163581791 0.32206221052666
-0.650325162581291 0.322904087768752
-0.646323161580791 0.323742980582282
-0.64232116058029 0.324578854307692
-0.63831915957979 0.325411674340905
-0.63431715857929 0.326241406135688
-0.63031515757879 0.327068015206018
-0.62631315657829 0.327891467128448
-0.622311155577789 0.328711727544471
-0.618309154577289 0.329528762162892
-0.614307153576789 0.330342536762189
-0.610305152576288 0.331153017192888
-0.606303151575788 0.331960169379928
-0.602301150575288 0.332763959325029
-0.598299149574788 0.333564353109062
-0.594297148574287 0.334361316894414
-0.590295147573787 0.335154816927355
-0.586293146573287 0.335944819540404
-0.582291145572787 0.336731291154694
-0.578289144572286 0.33751419828233
-0.574287143571786 0.338293507528757
-0.570285142571286 0.339069185595113
-0.566283141570786 0.339841199280588
-0.562281140570285 0.340609515484778
-0.558279139569785 0.341374101210036
-0.554277138569285 0.342134923563822
-0.550275137568784 0.342891949761048
-0.546273136568284 0.343645147126416
-0.542271135567784 0.344394483096766
-0.538269134567284 0.345139925223399
-0.534267133566784 0.34588144117442
-0.530265132566283 0.346618998737052
-0.526263131565783 0.347352565819966
-0.522261130565283 0.348082110455598
-0.518259129564782 0.348807600802455
-0.514257128564283 0.349529005147429
-0.510255127563782 0.350246291908092
-0.506253126563282 0.350959429634997
-0.502251125562782 0.351668387013963
-0.498249124562281 0.352373132868363
-0.494247123561781 0.353073636161397
-0.490245122561281 0.353769865998368
-0.486243121560781 0.354461791628937
-0.482241120560281 0.35514938244939
-0.47823911955978 0.355832608004878
-0.47423711855928 0.356511437991666
-0.47023511755878 0.35718584225936
-0.466233116558279 0.35785579081314
-0.462231115557779 0.358521253815973
-0.458229114557279 0.359182201590824
-0.454227113556779 0.359838604622853
-0.450225112556278 0.360490433561614
-0.446223111555778 0.36113765922323
-0.442221110555278 0.36178025259257
-0.438219109554778 0.362418184825411
-0.434217108554277 0.363051427250593
-0.430215107553777 0.36367995137216
-0.426213106553277 0.364303728871497
-0.422211105552777 0.364922731609448
-0.418209104552276 0.365536931628432
-0.414207103551776 0.366146301154544
-0.410205102551276 0.366750812599641
-0.406203101550775 0.367350438563426
-0.402201100550275 0.36794515183551
-0.398199099549775 0.368534925397471
-0.394197098549275 0.369119732424894
-0.390195097548775 0.369699546289402
-0.386193096548274 0.37027434056068
-0.382191095547774 0.37084408900847
-0.378189094547274 0.371408765604575
-0.374187093546773 0.371968344524831
-0.370185092546274 0.372522800151076
-0.366183091545773 0.373072107073103
-0.362181090545273 0.373616240090599
-0.358179089544773 0.374155174215069
-0.354177088544272 0.374688884671749
-0.350175087543772 0.375217346901498
-0.346173086543272 0.375740536562688
-0.342171085542772 0.376258429533063
-0.338169084542272 0.376771001911598
-0.334167083541771 0.377278230020329
-0.330165082541271 0.377780090406183
-0.326163081540771 0.378276559842778
-0.32216108054027 0.378767615332216
-0.31815907953977 0.379253234106858
-0.31415707853927 0.37973339363108
-0.31015507753877 0.380208071603017
-0.30615307653827 0.38067724595629
-0.302151075537769 0.381140894861712
-0.298149074537269 0.381598996728981
-0.294147073536769 0.382051530208356
-0.290145072536268 0.382498474192312
-0.286143071535768 0.382939807817184
-0.282141070535268 0.383375510464787
-0.278139069534768 0.383805561764022
-0.274137068534267 0.384229941592462
-0.270135067533767 0.384648630077921
-0.266133066533267 0.385061607600005
-0.262131065532766 0.385468854791645
-0.258129064532266 0.385870352540608
-0.254127063531766 0.386266081990992
-0.250125062531266 0.386656024544704
-0.246123061530766 0.387040161862913
-0.242121060530265 0.387418475867491
-0.238119059529765 0.387790948742426
-0.234117058529265 0.388157562935224
-0.230115057528764 0.388518301158288
-0.226113056528265 0.388873146390271
-0.222111055527764 0.38922208187742
-0.218109054527264 0.389565091134893
-0.214107053526764 0.389902157948054
-0.210105052526263 0.390233266373754
-0.206103051525763 0.390558400741584
-0.202101050525263 0.390877545655114
-0.198099049524763 0.391190685993109
-0.194097048524263 0.391497806910718
-0.190095047523762 0.391798893840651
-0.186093046523262 0.392093932494329
-0.182091045522762 0.392382908863015
-0.178089044522261 0.392665809218922
-0.174087043521761 0.392942620116299
-0.170085042521261 0.393213328392496
-0.166083041520761 0.39347792116901
-0.162081040520261 0.393736385852502
-0.15807903951976 0.393988710135799
-0.15407703851926 0.39423488199887
-0.15007503751876 0.394474889709777
-0.146073036518259 0.394708721825613
-0.142071035517759 0.394936367193407
-0.138069034517259 0.39515781495101
-0.134067033516759 0.395373054527962
-0.130065032516258 0.39558207564633
-0.126063031515758 0.395784868321529
-0.122061030515258 0.395981422863112
-0.118059029514757 0.396171729875547
-0.114057028514257 0.396355780258964
-0.110055027513757 0.396533565209877
-0.106053026513257 0.39670507622189
-0.102051025512757 0.396870305086375
-0.0980490245122563 0.397029243893121
-0.0940470235117563 0.397181885030975
-0.0900450225112559 0.397328221188439
-0.0860430215107555 0.397468245354264
-0.0820410205102555 0.397601950817999
-0.0780390195097551 0.397729331170536
-0.0740370185092547 0.397850380304619
-0.0700350175087547 0.39796509241533
-0.0660330165082543 0.398073462000554
-0.0620310155077544 0.398175483861422
-0.058029014507254 0.398271153102722
-0.0540270135067535 0.398360465133294
-0.0500250125062536 0.398443415666393
-0.0460230115057532 0.398520000720038
-0.0420210105052528 0.398590216617325
-0.0380190095047528 0.398654059986726
-0.0340170085042524 0.398711527762353
-0.030015007503752 0.398762617184209
-0.026013006503252 0.398807325798407
-0.0220110055027516 0.398845651457363
-0.0180090045022516 0.398877592319974
-0.0140070035017512 0.398903146851762
-0.0100050025012508 0.398922313824996
-0.00600300150075084 0.398935092318793
-0.00200100050025043 0.398941481719188
0.00200100050024954 0.398941481719188
0.00600300150075039 0.398935092318793
0.0100050025012504 0.398922313824996
0.0140070035017503 0.398903146851762
0.0180090045022512 0.398877592319974
0.0220110055027511 0.398845651457363
0.0260130065032511 0.398807325798407
0.030015007503752 0.398762617184209
0.0340170085042519 0.398711527762353
0.0380190095047519 0.398654059986726
0.0420210105052528 0.398590216617325
0.0460230115057527 0.398520000720038
0.0500250125062527 0.398443415666393
0.0540270135067527 0.398360465133294
0.0580290145072535 0.398271153102722
0.0620310155077535 0.398175483861422
0.0660330165082534 0.398073462000554
0.0700350175087543 0.39796509241533
0.0740370185092543 0.397850380304619
0.0780390195097542 0.397729331170536
0.0820410205102551 0.397601950817999
0.086043021510755 0.397468245354264
0.090045022511255 0.39732822118844
0.0940470235117559 0.397181885030975
0.0980490245122558 0.397029243893121
0.102051025512756 0.396870305086375
0.106053026513257 0.39670507622189
0.110055027513757 0.396533565209877
0.114057028514257 0.396355780258964
0.118059029514757 0.396171729875547
0.122061030515257 0.395981422863112
0.126063031515757 0.395784868321529
0.130065032516258 0.39558207564633
0.134067033516758 0.395373054527962
0.138069034517258 0.39515781495101
0.142071035517759 0.394936367193407
0.146073036518259 0.394708721825613
0.150075037518759 0.394474889709777
0.154077038519259 0.39423488199887
0.15807903951976 0.393988710135799
0.16208104052026 0.393736385852502
0.16608304152076 0.39347792116901
0.170085042521261 0.393213328392496
0.174087043521761 0.392942620116299
0.17808904452226 0.392665809218922
0.182091045522761 0.392382908863015
0.186093046523261 0.392093932494329
0.190095047523761 0.391798893840651
0.194097048524262 0.391497806910718
0.198099049524762 0.391190685993109
0.202101050525262 0.390877545655115
0.206103051525763 0.390558400741584
0.210105052526263 0.390233266373754
0.214107053526763 0.389902157948054
0.218109054527264 0.389565091134893
0.222111055527764 0.38922208187742
0.226113056528264 0.388873146390271
0.230115057528764 0.388518301158288
0.234117058529264 0.388157562935224
0.238119059529764 0.387790948742426
0.242121060530264 0.387418475867491
0.246123061530765 0.387040161862913
0.250125062531265 0.386656024544704
0.254127063531765 0.386266081990992
0.258129064532266 0.385870352540608
0.262131065532766 0.385468854791645
0.266133066533266 0.385061607600005
0.270135067533767 0.384648630077921
0.274137068534267 0.384229941592462
0.278139069534767 0.383805561764022
0.282141070535268 0.383375510464787
0.286143071535768 0.382939807817184
0.290145072536268 0.382498474192312
0.294147073536768 0.382051530208356
0.298149074537268 0.381598996728981
0.302151075537768 0.381140894861712
0.306153076538269 0.38067724595629
0.310155077538769 0.380208071603017
0.314157078539269 0.37973339363108
0.31815907953977 0.379253234106858
0.32216108054027 0.378767615332216
0.32616308154077 0.378276559842778
0.330165082541271 0.377780090406183
0.334167083541771 0.377278230020329
0.338169084542271 0.376771001911598
0.342171085542771 0.376258429533064
0.346173086543272 0.375740536562688
0.350175087543771 0.375217346901498
0.354177088544271 0.374688884671749
0.358179089544772 0.374155174215069
0.362181090545272 0.3736162400906
0.366183091545772 0.373072107073104
0.370185092546273 0.372522800151076
0.374187093546773 0.371968344524831
0.378189094547273 0.371408765604575
0.382191095547774 0.37084408900847
0.386193096548274 0.37027434056068
0.390195097548774 0.369699546289403
0.394197098549275 0.369119732424894
0.398199099549775 0.368534925397471
0.402201100550275 0.367945151835511
0.406203101550775 0.367350438563426
0.410205102551275 0.366750812599641
0.414207103551775 0.366146301154544
0.418209104552276 0.365536931628432
0.422211105552776 0.364922731609448
0.426213106553276 0.364303728871497
0.430215107553777 0.36367995137216
0.434217108554277 0.363051427250593
0.438219109554777 0.362418184825412
0.442221110555277 0.36178025259257
0.446223111555778 0.36113765922323
0.450225112556278 0.360490433561614
0.454227113556778 0.359838604622853
0.458229114557279 0.359182201590824
0.462231115557779 0.358521253815973
0.466233116558278 0.357855790813141
0.470235117558779 0.35718584225936
0.474237118559279 0.356511437991666
0.478239119559779 0.355832608004878
0.48224112056028 0.35514938244939
0.48624312156078 0.354461791628937
0.49024512256128 0.353769865998368
0.494247123561781 0.353073636161397
0.498249124562281 0.352373132868363
0.502251125562781 0.351668387013963
0.506253126563282 0.350959429634997
0.510255127563782 0.350246291908092
0.514257128564282 0.349529005147429
0.518259129564782 0.348807600802455
0.522261130565282 0.348082110455598
0.526263131565782 0.347352565819966
0.530265132566282 0.346618998737052
0.534267133566783 0.34588144117442
0.538269134567283 0.3451399252234
0.542271135567783 0.344394483096766
0.546273136568284 0.343645147126416
0.550275137568784 0.342891949761048
0.554277138569284 0.342134923563823
0.558279139569785 0.341374101210036
0.562281140570285 0.340609515484778
0.566283141570785 0.339841199280588
0.570285142571286 0.339069185595113
0.574287143571786 0.338293507528757
0.578289144572286 0.33751419828233
0.582291145572786 0.336731291154694
0.586293146573286 0.335944819540404
0.590295147573786 0.335154816927355
0.594297148574287 0.334361316894414
0.598299149574787 0.333564353109062
0.602301150575287 0.33276395932503
0.606303151575788 0.331960169379928
0.610305152576288 0.331153017192888
0.614307153576788 0.330342536762189
0.618309154577289 0.329528762162892
0.622311155577789 0.328711727544471
0.626313156578289 0.327891467128448
0.630315157578789 0.327068015206018
0.634317158579289 0.326241406135688
0.638319159579789 0.325411674340905
0.642321160580289 0.324578854307693
0.64632316158079 0.323742980582282
0.65032516258129 0.322904087768752
0.65432716358179 0.32206221052666
0.658329164582291 0.321217383568686
0.662331165582791 0.320369641658273
0.666333166583291 0.319519019607265
0.670335167583792 0.318665552273557
0.674337168584292 0.31780927455874
0.678339169584792 0.316950221405752
0.682341170585293 0.316088427796532
0.686343171585793 0.315223928749676
0.690345172586293 0.314356759318096
0.694347173586793 0.313486954586685
0.698349174587293 0.312614549669984
0.702351175587793 0.311739579709853
0.706353176588294 0.310862079873149
0.710355177588794 0.309982085349404
0.714357178589294 0.309099631348509
0.718359179589794 0.308214753098406
0.722361180590295 0.307327485842785
0.726363181590795 0.306437864838779
0.730365182591295 0.305545925354673
0.734367183591796 0.304651702667618
0.738369184592296 0.303755232061342
0.742371185592796 0.30285654882388
0.746373186593297 0.301955688245302
0.750375187593797 0.301052685615446
0.754377188594296 0.300147576221666
0.758379189594797 0.299240395346582
0.762381190595297 0.298331178265834
0.766383191595797 0.297419960245848
0.770385192596298 0.29650677654161
0.774387193596798 0.295591662394443
0.778389194597298 0.294674653029797
0.782391195597799 0.293755783655043
0.786393196598299 0.292835089457281
0.790395197598799 0.291912605601146
0.7943971985993 0.290988367226634
0.7983991995998 0.290062409446931
0.8024012006003 0.289134767346251
0.8064032016008 0.288205475977687
0.8104052026013 0.287274570361064
0.8144072036018 0.286342085480809
0.8184092046023 0.285408056283828
0.822411205602801 0.284472517677391
0.826413206603301 0.283535504527029
0.830415207603801 0.282597051654442
0.834417208604302 0.281657193835417
0.838419209604802 0.280715965797751
0.842421210605302 0.279773402219196
0.846423211605803 0.278829537725403
0.850425212606303 0.277884406887888
0.854427213606803 0.276938044221996
0.858429214607304 0.275990484184891
0.862431215607804 0.275041761173547
0.866433216608304 0.274091909522756
0.870435217608804 0.273140963503143
0.874437218609304 0.272188957319199
0.878439219609804 0.271235925107322
0.882441220610305 0.270281900933869
0.886443221610805 0.269326918793228
0.890445222611305 0.26837101260589
0.894447223611806 0.267414216216547
0.898449224612306 0.266456563392195
0.902451225612806 0.265498087820249
0.906453226613307 0.264538823106676
0.910455227613807 0.263578802774142
0.914457228614307 0.26261806026016
0.918459229614807 0.261656628915273
0.922461230615307 0.260694542001228
0.926463231615807 0.259731832689179
0.930465232616307 0.2587685340579
0.934467233616808 0.257804679092007
0.938469234617308 0.2568403006802
0.942471235617808 0.255875431613521
0.946473236618309 0.254910104583613
0.950475237618809 0.253944352181015
0.954477238619309 0.252978206893449
0.95847923961981 0.252011701104139
0.96248124062031 0.251044867090135
0.96648324162081 0.250077737020655
0.970485242621311 0.249110342955442
0.974487243621811 0.248142716843138
0.978489244622311 0.247174890519666
0.982491245622811 0.246206895706638
0.986493246623311 0.245238764009768
0.990495247623811 0.244270526917309
0.994497248624312 0.243302215798498
0.998499249624812 0.242333861902022
1.00250125062531 0.241365496354499
1.00650325162581 0.240397150158971
1.01050525262631 0.239428854193418
1.01450725362681 0.238460639209285
1.01850925462731 0.237492535830026
1.02251125562781 0.23652457454966
1.02651325662831 0.235556785731354
1.03051525762881 0.234589199606009
1.03451725862931 0.233621846270869
1.03851925962981 0.23265475568815
1.04252126063031 0.231687957683679
1.04652326163082 0.230721481945548
1.05052526263132 0.229755358022795
1.05452726363182 0.228789615324091
1.05852926463232 0.227824283116448
1.06253126563282 0.226859390523945
1.06653326663332 0.225894966526467
1.07053526763382 0.224931039958466
1.07453726863432 0.223967639507733
1.07853926963482 0.22300479371419
1.08254127063532 0.222042530968703
1.08654327163582 0.221080879511904
1.09054527263632 0.220119867433033
1.09454727363682 0.219159522668802
1.09854927463732 0.21819987300227
1.10255127563782 0.217240946061737
1.10655327663832 0.216282769319657
1.11055527763882 0.215325370091563
1.11455727863932 0.214368775535022
1.11855927963982 0.213413012648586
1.12256128064032 0.212458108270781
1.12656328164082 0.211504089079106
1.13056528264132 0.210550981589042
1.13456728364182 0.209598812153087
1.13856928464232 0.20864760695981
1.14257128564282 0.207697392032913
1.14657328664332 0.206748193230319
1.15057528764382 0.205800036243273
1.15457728864432 0.204852946595463
1.15857928964482 0.203906949642153
1.16258129064532 0.202962070569344
1.16658329164582 0.202018334392941
1.17058529264632 0.201075765957943
1.17458729364682 0.200134389937651
1.17858929464732 0.199194230832891
1.18259129564782 0.198255312971257
1.18659329664832 0.197317660506369
1.19059529764882 0.196381297417147
1.19459729864932 0.195446247507111
1.19859929964982 0.194512534403684
1.20260130065032 0.193580181557527
1.20660330165082 0.192649212241883
1.21060530265133 0.191719649551935
1.21460730365183 0.190791516404196
1.21860930465233 0.189864835535897
1.22261130565283 0.188939629504407
1.22661330665333 0.188015920686666
1.23061530765383 0.187093731278629
1.23461730865433 0.186173083294738
1.23861930965483 0.185253998567401
1.24262131065533 0.184336498746495
1.24662331165583 0.183420605298883
1.25062531265633 0.182506339507951
1.25462731365683 0.181593722473152
1.25862931465733 0.180682775109584
1.26263131565783 0.179773518147568
1.26663331665833 0.178865972132256
1.27063531765883 0.177960157423245
1.27463731865933 0.177056094194214
1.27863931965983 0.17615380243258
1.28264132066033 0.175253301939161
1.28664332166083 0.174354612327867
1.29064532266133 0.1734577530254
1.29464732366183 0.172562743270971
1.29864932466233 0.171669602116037
1.30265132566283 0.170778348424055
1.30665332666333 0.169889000870244
1.31065532766383 0.169001577941371
1.31465732866433 0.168116097935554
1.31865932966483 0.167232578962077
1.32266133066533 0.166351038941219
1.32666333166583 0.165471495604106
1.33066533266633 0.164593966492573
1.33466733366683 0.163718468959048
1.33866933466733 0.16284502016644
1.34267133566783 0.16197363708806
1.34667333666833 0.161104336507539
1.35067533766883 0.160237135018777
1.35467733866933 0.1593720490259
1.35867933966983 0.158509094743231
1.36268134067033 0.157648288195279
1.36668334167083 0.156789645216746
1.37068534267134 0.155933181452544
1.37468734367184 0.155078912357829
1.37868934467234 0.15422685319805
1.38269134567284 0.153377019049015
1.38669334667334 0.152529424796967
1.39069534767384 0.15168408513868
1.39469734867434 0.150841014581564
1.39869934967484 0.150000227443793
1.40270135067534 0.149161737854435
1.40670335167584 0.148325559753611
1.41070535267634 0.147491706892653
1.41470735367684 0.14666019283429
1.41870935467734 0.145831030952839
1.42271135567784 0.145004234434413
1.42671335667834 0.144179816277144
1.43071535767884 0.143357789291416
1.43471735867934 0.142538166100115
1.43871935967984 0.141720959138896
1.44272136068034 0.14090618065645
1.44672336168084 0.140093842714802
1.45072536268134 0.13928395718961
1.45472736368184 0.13847653577048
1.45872936468234 0.137671589961297
1.46273136568284 0.136869131080565
1.46673336668334 0.136069170261763
1.47073536768384 0.135271718453711
1.47473736868434 0.134476786420952
1.47873936968484 0.133684384744142
1.48274137068534 0.132894523820455
1.48674337168584 0.132107213864001
1.49074537268634 0.131322464906257
1.49474737368684 0.130540286796504
1.49874937468734 0.129760689202283
1.50275137568784 0.12898368160986
1.50675337668834 0.128209273324702
1.51075537768884 0.127437473471964
1.51475737868934 0.126668290996991
1.51875937968984 0.125901734665829
1.52276138069035 0.125137813065745
1.52676338169085 0.124376534605763
1.53076538269134 0.123617907517205
1.53476738369185 0.122861939854253
1.53876938469235 0.122108639494507
1.54277138569285 0.121358014139567
1.54677338669335 0.120610071315618
1.55077538769385 0.119864818374029
1.55477738869435 0.119122262491959
1.55877938969485 0.118382410672978
1.56278139069535 0.117645269747692
1.56678339169585 0.116910846374384
1.57078539269635 0.116179147039658
1.57478739369685 0.1154501780591
1.57878939469735 0.114723945577943
1.58279139569785 0.114000455571746
1.58679339669835 0.113279713847074
1.59079539769885 0.112561726042201
1.59479739869935 0.111846497627806
1.59879939969985 0.111134033907692
1.60280140070035 0.110424340019505
1.60680340170085 0.109717420935463
1.61080540270135 0.109013281463097
1.61480740370185 0.108311926245998
1.61880940470235 0.107613359764572
1.62281140570285 0.106917586336804
1.62681340670335 0.10622461011903
1.63081540770385 0.105534435106718
1.63481740870435 0.10484706513525
1.63881940970485 0.104162503880726
1.64282141070535 0.103480754860759
1.64682341170585 0.102801821435289
1.65082541270635 0.1021257068074
1.65482741370685 0.101452414024144
1.65882941470735 0.100781945977373
1.66283141570785 0.100114305404576
1.66683341670835 0.0994494948897266
1.67083541770885 0.0987875168641352
1.67483741870935 0.098128373607305
1.67883941970985 0.097472067247799
1.68284142071035 0.0968185997641101
1.68684342171086 0.0961679729855392
1.69084542271136 0.0955201885930786
1.69484742371186 0.0948752481203005
1.69884942471236 0.0942331529542529
1.70285142571286 0.0935939043363603
1.70685342671336 0.0929575033633295
1.71085542771386 0.0923239509880615
1.71485742871436 0.091693248020569
1.71885942971486 0.0910653951288976
1.72286143071536 0.0904403928400537
1.72686343171586 0.0898182415409364
1.73086543271636 0.0891989414792742
1.73486743371686 0.0885824927645667
1.73886943471736 0.0879688953690308
1.74287143571786 0.0873581491285511
1.74687343671836 0.0867502537436344
1.75087543771886 0.0861452087803698
1.75487743871936 0.0855430136713901
1.75887943971986 0.0849436677168402
1.76288144072036 0.0843471700853476
1.76688344172086 0.0837535198149965
1.77088544272136 0.0831627158143062
1.77488744372186 0.0825747568632126
1.77888944472236 0.0819896416140535
1.78289144572286 0.0814073685925562
1.78689344672336 0.0808279361988287
1.79089544772386 0.0802513427083549
1.79489744872436 0.0796775862729904
1.79889944972486 0.0791066649219633
1.80290145072536 0.0785385765628764
1.80690345172586 0.0779733189827125
1.81090545272636 0.077410889848841
1.81490745372686 0.0768512867100294
1.81890945472736 0.0762945069974532
1.82291145572786 0.0757405480257113
1.82691345672836 0.0751894069938419
1.83091545772886 0.0746410809863396
1.83491745872936 0.0740955669741756
1.83891945972986 0.0735528618158191
1.84292146073036 0.0730129622582593
1.84692346173087 0.0724758649380299
1.85092546273137 0.0719415663822351
1.85492746373187 0.0714100630095754
1.85892946473237 0.0708813511313756
1.86293146573287 0.0703554269526141
1.86693346673337 0.0698322865729515
1.87093546773387 0.0693119259877619
1.87493746873437 0.0687943410891631
1.87893946973487 0.068279527667049
1.88294147073537 0.0677674814101211
1.88694347173587 0.0672581979069211
1.89094547273637 0.0667516726468636
1.89494747373687 0.0662479010212689
1.89894947473737 0.0657468783243956
1.90295147573787 0.0652485997544744
1.90695347673837 0.0647530604147394
1.91095547773887 0.0642602553144615
1.91495747873937 0.0637701793699806
1.91895947973987 0.0632828274057366
1.92296148074037 0.0627981941553008
1.92696348174087 0.0623162742624073
1.93096548274137 0.0618370622819814
1.93496748374187 0.0613605526811695
1.93896948474237 0.0608867398403671
1.94297148574287 0.0604156180542459
1.94697348674337 0.0599471815327793
1.95097548774387 0.0594814244022684
1.95497748874437 0.0590183407063642
1.95897948974487 0.058557924407091
1.96298149074537 0.0581001693858663
1.96698349174587 0.0576450694445211
1.97098549274637 0.0571926183063164
1.97498749374687 0.0567428096169598
1.97898949474737 0.0562956369456199
1.98299149574787 0.0558510937859381
1.98699349674837 0.0554091735570391
1.99099549774887 0.0549698696045397
1.99499749874937 0.0545331752015544
1.99899949974987 0.0540990835496997
2.00300150075037 0.0536675877800962
2.00700350175087 0.0532386809543676
2.01100550275138 0.0528123560656379
2.01500750375188 0.0523886060395261
2.01900950475238 0.0519674237351378
2.02301150575288 0.0515488019460546
2.02701350675338 0.0511327334013212
2.03101550775388 0.0507192107664285
2.03501750875438 0.0503082266442947
2.03901950975488 0.0498997735762438
2.04302151075538 0.04949384404298
2.04702351175588 0.0490904304655595
2.05102551275638 0.0486895252063604
2.05502751375688 0.0482911205700468
2.05902951475738 0.0478952088045321
2.06303151575788 0.0475017821019374
2.06703351675838 0.0471108325995477
2.07103551775888 0.0467223523807631
2.07503751875938 0.0463363334760482
2.07903951975988 0.0459527678638764
2.08304152076038 0.0455716474716718
2.08704352176088 0.045192964176746
2.09104552276138 0.0448167098072329
2.09504752376188 0.0444428761430179
2.09904952476238 0.0440714549166642
2.10305152576288 0.0437024378143354
2.10705352676338 0.043335816476713
2.11105552776388 0.0429715824999109
2.11505752876438 0.0426097274363853
2.11905952976488 0.0422502427958405
2.12306153076538 0.0418931200461305
2.12706353176588 0.0415383506141568
2.13106553276638 0.0411859258867606
2.13506753376688 0.0408358372116122
2.13906953476738 0.040488075898095
2.14307153576788 0.0401426332181855
2.14707353676838 0.0397995004073283
2.15107553776888 0.0394586686653071
2.15507753876938 0.0391201291571111
2.15907953976988 0.038783873013796
2.16308154077038 0.0384498913333411
2.16708354177088 0.0381181751815019
2.17108554277138 0.0377887155926567
2.17508754377189 0.0374615035706496
2.17908954477239 0.0371365300896286
2.18309154577289 0.0368137860948775
2.18709354677339 0.0364932625036447
2.19109554777389 0.0361749502059659
2.19509754877439 0.035858840065482
2.19909954977489 0.0355449229202521
2.20310155077539 0.0352331895835617
2.20710355177589 0.0349236308447252
2.21110555277639 0.0346162374698836
2.21510755377689 0.0343110002027972
2.21910955477739 0.0340079097656332
2.22311155577789 0.033706956859747
2.22711355677839 0.0334081321664603
2.23111555777889 0.0331114263478317
2.23511755877939 0.0328168300474235
2.23911955977989 0.0325243338910628
2.24312156078039 0.0322339284875969
2.24712356178089 0.0319456044296436
2.25112556278139 0.0316593522943363
2.25512756378189 0.0313751626440634
2.25912956478239 0.031093026027202
2.26313156578289 0.0308129329788468
2.26713356678339 0.0305348740215333
2.27113556778389 0.0302588396659547
2.27513756878439 0.0299848204116746
2.27913956978489 0.0297128067478332
2.28314157078539 0.0294427891538483
2.28714357178589 0.0291747581001104
2.29114557278639 0.0289087040486729
2.29514757378689 0.0286446174539358
2.29914957478739 0.0283824887633247
2.30315157578789 0.0281223084179631
2.30715357678839 0.0278640668533402
2.31115557778889 0.0276077544999719
2.31515757878939 0.0273533617840575
2.31915957978989 0.027100879128129
2.3231615807904 0.0268502969516961
2.3271635817909 0.0266016056718852
2.3311655827914 0.0263547957040721
2.3351675837919 0.0261098574625096
2.3391695847924 0.0258667813609493
2.3431715857929 0.0256255578132569
2.3471735867934 0.0253861772340234
2.3511755877939 0.0251486300391685
2.3551775887944 0.0249129066465401
2.3591795897949 0.0246789974765069
2.3631815907954 0.0244468929525454
2.3671835917959 0.0242165835018219
2.3711855927964 0.0239880595557676
2.3751875937969 0.0237613115506488
2.3791895947974 0.0235363299281312
2.3831915957979 0.0233131051358379
2.3871935967984 0.0230916276279024
2.3911955977989 0.0228718878655155
2.3951975987994 0.0226538763174659
2.3991995997999 0.0224375834606764
2.4032016008004 0.022222999780733
2.4072036018009 0.0220101157724089
2.4112056028014 0.0217989219401825
2.4152076038019 0.0215894087987504
2.4192096048024 0.0213815668735334
2.4232116058029 0.0211753867011778
2.4272136068034 0.0209708588300509
2.4312156078039 0.0207679738207301
2.4352176088044 0.0205667222464868
2.4392196098049 0.0203670946937649
2.4432216108054 0.0201690817626533
2.4472236118059 0.0199726740673523
2.4512256128064 0.0197778622366349
2.4552276138069 0.0195846369143029
2.4592296148074 0.0193929887596358
2.4632316158079 0.0192029084478357
2.4672336168084 0.0190143866704659
2.4712356178089 0.0188274141358837
2.4752376188094 0.018641981569668
2.4792396198099 0.0184580797150413
2.4832416208104 0.0182756993332856
2.48724362181091 0.0180948312041537
2.49124562281141 0.0179154661262742
2.49524762381191 0.0177375949175511
2.49924962481241 0.0175612084155583
2.50325162581291 0.0173862974779283
2.50725362681341 0.0172128529827356
2.51125562781391 0.0170408658288742
2.51525762881441 0.0168703269364303
2.51925962981491 0.0167012272470496
2.52326163081541 0.0165335577242984
2.52726363181591 0.0163673093540201
2.53126563281641 0.0162024731446865
2.53526763381691 0.0160390401277425
2.53926963481741 0.0158770013579475
2.54327163581791 0.0157163479137096
2.54727363681841 0.0155570708974155
2.55127563781891 0.0153991614357553
2.55527763881941 0.0152426106800413
2.55927963981991 0.0150874098065225
2.56328164082041 0.0149335500166932
2.56728364182091 0.0147810225375969
2.57128564282141 0.0146298186221249
2.57528764382191 0.0144799295493096
2.57928964482241 0.0143313466246132
2.58329164582291 0.0141840611802109
2.58729364682341 0.0140380645752692
2.59129564782391 0.0138933481962194
2.59529764882441 0.0137499034570256
2.59929964982491 0.0136077217994486
2.60330165082541 0.0134667946933038
2.60730365182591 0.013327113636715
2.61130565282641 0.0131886701563631
2.61530765382691 0.0130514558077297
2.61930965482741 0.0129154621753365
2.62331165582791 0.0127806808729789
2.62731365682841 0.0126471035439559
2.63131565782891 0.0125147218612947
2.63531765882941 0.0123835275279704
2.63931965982991 0.0122535122771213
2.64332166083041 0.0121246678722601
2.64732366183091 0.0119969861074791
2.65132566283142 0.0118704588076518
2.65532766383192 0.0117450778286303
2.65932966483242 0.0116208350574366
2.66333166583292 0.0114977224124512
2.66733366683342 0.011375731843596
2.67133566783392 0.011254855332513
2.67533766883442 0.011135084892739
2.67933966983492 0.0110164125698756
2.68334167083542 0.0108988304417546
2.68734367183592 0.0107823306185994
2.69134567283642 0.0106669052431821
2.69534767383692 0.0105525464909763
2.69934967483742 0.0104392465703055
2.70335167583792 0.0103269977224873
2.70735367683842 0.0102157922219739
2.71135567783892 0.0101056223764876
2.71535767883942 0.009996480527153
2.71935967983992 0.00988835904862463
2.72336168084042 0.00978125034921079
2.72736368184092 0.00967514687099317
2.73136568284142 0.00957004108994274
2.73536768384192 0.00946592551603128
2.73936968484242 0.00936279269333946
2.74337168584292 0.00926063520016073
2.74737368684342 0.0091594456491014
2.75137568784392 0.00905921668717695
2.75537768884442 0.00895994099590459
2.75937968984492 0.0088616112913919
2.76338169084542 0.00876422032442184
2.76738369184592 0.00866776088053422
2.77138569284642 0.00857222578010315
2.77538769384692 0.00847760787841116
2.77938969484742 0.00838390006571972
2.78339169584792 0.00829109526733594
2.78739369684842 0.00819918644367601
2.79139569784892 0.00810816659032508
2.79539769884942 0.00801802873809351
2.79939969984992 0.00792876595306993
2.80340170085042 0.00784037133667081
2.80740370185092 0.00775283802568652
2.81140570285143 0.00766615919232429
2.81540770385193 0.00758032804424777
2.81940970485243 0.00749533782461326
2.82341170585293 0.0074111818121028
2.82741370685343 0.00732785332095405
2.83141570785393 0.00724534570098698
2.83541770885443 0.0071636523376274
2.83941970985493 0.00708276665192743
2.84342171085543 0.00700268210058294
2.84742371185593 0.00692339217594784
2.85142571285643 0.00684489040604547
2.85542771385693 0.00676717035457704
2.85942971485743 0.00669022562092696
2.86343171585793 0.00661404984016549
2.86743371685843 0.00653863668304844
2.87143571785893 0.00646397985601389
2.87543771885943 0.00639007310117632
2.87943971985993 0.0063169101963179
2.88344172086043 0.00624448495487691
2.88744372186093 0.00617279122593363
2.89144572286143 0.0061018228941935
2.89544772386193 0.00603157387996754
2.89944972486243 0.00596203813915028
2.90345172586293 0.00589320966319514
2.90745372686343 0.00582508247908702
2.91145572786393 0.00575765064931272
2.91545772886443 0.00569090827182859
2.91945972986493 0.00562484948002594
2.92346173086543 0.00555946844269385
2.92746373186593 0.00549475936397967
2.93146573286643 0.00543071648334728
2.93546773386693 0.00536733407553281
2.93946973486743 0.00530460645049819
2.94347173586793 0.00524252795338248
2.94747373686843 0.00518109296445081
2.95147573786893 0.00512029589904129
2.95547773886943 0.00506013120750969
2.95947973986993 0.00500059337517191
2.96348174087044 0.00494167692224441
2.96748374187093 0.00488337640378264
2.97148574287143 0.00482568640961719
2.97548774387194 0.00476860156428819
2.97948974487244 0.00471211652697757
2.98349174587294 0.00465622599143935
2.98749374687344 0.00460092468592809
2.99149574787394 0.00454620737312541
2.99549774887444 0.00449206885006459
2.99949974987494 0.00443850394805337
3.00350175087544 0.00438550753259502
3.00750375187594 0.00433307450330744
3.01150575287644 0.00428119979384071
3.01550775387694 0.00422987837179276
3.01950975487744 0.0041791052386235
3.02351175587794 0.00412887542956708
3.02751375687844 0.00407918401354273
3.03151575787894 0.00403002609306387
3.03551775887944 0.00398139680414561
3.03951975987994 0.00393329131621079
3.04352176088044 0.00388570483199447
3.04752376188094 0.00383863258744685
3.05152576288144 0.00379206985163481
3.05552776388194 0.00374601192664198
3.05952976488244 0.00370045414746733
3.06353176588294 0.00365539188192245
3.06753376688344 0.00361082053052751
3.07153576788394 0.00356673552640569
3.07553776888444 0.00352313233517649
3.07953976988494 0.00348000645484773
3.08354177088544 0.00343735341570616
3.08754377188594 0.00339516878020699
3.09154577288644 0.00335344814286219
3.09554777388694 0.00331218713012746
3.09954977488744 0.00327138140028826
3.10355177588794 0.00323102664334457
3.10755377688844 0.00319111858089459
3.11155577788894 0.00315165296601732
3.11555777888944 0.00311262558315415
3.11955977988994 0.00307403224798939
3.12356178089044 0.00303586880732975
3.12756378189095 0.00299813113898286
3.13156578289145 0.00296081515163491
3.13556778389195 0.00292391678472723
3.13956978489245 0.00288743200833198
3.14357178589295 0.00285135682302709
3.14757378689345 0.00281568725977011
3.15157578789395 0.00278041937977138
3.15557778889445 0.00274554927436635
3.15957978989495 0.00271107306488705
3.16358179089545 0.00267698690253281
3.16758379189595 0.00264328696824025
3.17158579289645 0.00260996947255253
3.17558779389695 0.0025770306554878
3.17958979489745 0.00254446678640717
3.18359179589795 0.00251227416388175
3.18759379689845 0.00248044911555922
3.19159579789895 0.00244898799802977
3.19559779889945 0.00241788719669132
3.19959979989995 0.00238714312561428
3.20360180090045 0.00235675222740563
3.20760380190095 0.00232671097307263
3.21160580290145 0.00229701586188575
3.21560780390195 0.00226766342124135
3.21960980490245 0.00223865020652371
3.22361180590295 0.00220997280096666
3.22761380690345 0.00218162781551477
3.23161580790395 0.00215361188868407
3.23561780890445 0.00212592168642239
3.23961980990495 0.0020985539019693
3.24362181090545 0.00207150525571571
3.24762381190595 0.00204477249506308
3.25162581290645 0.00201835239428227
3.25562781390695 0.00199224175437216
3.25962981490745 0.00196643740291786
3.26363181590795 0.00194093619394875
3.26763381690845 0.00191573500779617
3.27163581790895 0.00189083075095088
3.27563781890945 0.0018662203559203
3.27963981990995 0.00184190078108557
3.28364182091045 0.00181786901055833
3.28764382191096 0.00179412205403736
3.29164582291146 0.00177065694666512
3.29564782391196 0.00174747074888397
3.29964982491246 0.00172456054629245
3.30365182591296 0.00170192344950132
3.30765382691346 0.00167955659398951
3.31165582791396 0.00165745713996004
3.31565782891446 0.00163562227219576
3.31965982991496 0.00161404919991516
3.32366183091546 0.00159273515662804
3.32766383191596 0.00157167739999117
3.33166583291646 0.00155087321166397
3.33566783391696 0.00153031989716415
3.33966983491746 0.00151001478572335
3.34367183591796 0.00148995523014288
3.34767383691846 0.0014701386066494
3.35167583791896 0.00145056231475072
3.35567783891946 0.00143122377709165
3.35967983991996 0.00141212043930991
3.36368184092046 0.00139324976989214
3.36768384192096 0.00137460926003005
3.37168584292146 0.00135619642347661
3.37568784392196 0.00133800879640242
3.37968984492246 0.00132004393725223
3.38369184592296 0.00130229942660157
3.38769384692346 0.00128477286701359
3.39169584792396 0.00126746188289601
3.39569784892446 0.00125036412035834
3.39969984992496 0.00123347724706921
3.40370185092546 0.00121679895211396
3.40770385192596 0.00120032694585249
3.41170585292646 0.00118405895977723
3.41570785392696 0.0011679927463714
3.41970985492746 0.0011521260789676
3.42371185592796 0.00113645675160653
3.42771385692846 0.00112098257889603
3.43171585792896 0.00110570139587049
3.43571785892946 0.00109061105785036
3.43971985992996 0.00107570944030215
3.44372186093046 0.00106099443869865
3.44772386193096 0.00104646396837941
3.45172586293147 0.0010321159644117
3.45572786393197 0.00101794838145165
3.45972986493247 0.00100395919360583
3.46373186593297 0.000990146394293136
3.46773386693347 0.000976507996107092
3.47173586793397 0.000963042030678431
3.47573786893447 0.000949746548538137
3.47973986993497 0.000936619618980838
3.48374187093547 0.00092365932992858
3.48774387193597 0.000910863787795027
3.49174587293647 0.000898231117350048
3.49574787393697 0.000885759461584739
3.49974987493747 0.000873446981576842
3.50375187593797 0.000861291856356611
3.50775387693847 0.00084929228277312
3.51175587793897 0.000837446475360983
3.51575787893947 0.000825752666207559
3.51975987993997 0.000814209104820594
3.52376188094047 0.000802814057996322
3.52776388194097 0.00079156580968804
3.53176588294147 0.000780462660875171
3.53576788394197 0.00076950292943277
3.53976988494247 0.00075868495000155
3.54377188594297 0.000748007073858396
3.54777388694347 0.000737467668787346
3.55177588794397 0.000727065118951111
3.55577788894447 0.000716797824763083
3.55977988994497 0.000706664202759852
3.56378189094547 0.000696662685474251
3.56778389194597 0.000686791721308924
3.57178589294647 0.000677049774410399
3.57578789394697 0.000667435324543723
3.57978989494747 0.000657946866967616
3.58379189594797 0.000648582912310158
3.58779389694847 0.000639341986445037
3.59179589794897 0.000630222630368331
3.59579789894947 0.000621223400075861
3.59979989994997 0.000612342866441069
3.60380190095047 0.000603579615093484
3.60780390195097 0.000594932246297751
3.61180590295147 0.000586399374833195
3.61580790395198 0.000577979629873994
3.61980990495248 0.000569671654869908
3.62381190595298 0.000561474107427577
3.62781390695348 0.000553385659192414
3.63181590795398 0.000545404995731075
3.63581790895448 0.000537530816414507
3.63981990995498 0.000529761834301592
3.64382191095548 0.000522096776023388
3.64782391195598 0.000514534381667945
3.65182591295648 0.000507073404665731
3.65582791395698 0.000499712611675658
3.65982991495748 0.000492450782471692
3.66383191595798 0.00048528670983008
3.66783391695848 0.000478219199417186
3.67183591795898 0.000471247069677908
3.67583791895948 0.000464369151724732
3.67983991995998 0.000457584289227377
3.68384192096048 0.000450891338303063
3.68784392196098 0.00044428916740738
3.69184592296148 0.000437776657225783
3.69584792396198 0.000431352700565701
3.69984992496248 0.000425016202249249
3.70385192596298 0.00041876607900658
3.70785392696348 0.000412601259369843
3.71185592796398 0.000406520683567761
3.71585792896448 0.000400523303420835
3.71985992996498 0.000394608082237174
3.72386193096548 0.000388773994708933
3.72786393196598 0.00038302002680939
3.73186593296648 0.000377345175690644
3.73586793396698 0.000371748449581929
3.73986993496748 0.000366228867688561
3.74387193596798 0.000360785460091515
3.74787393696848 0.000355417267647615
3.75187593796898 0.000350123341890356
3.75587793896948 0.000344902744931365
3.75987993996998 0.000339754549362461
3.76388194097049 0.000334677838158367
3.76788394197099 0.000329671704580039
3.77188594297149 0.000324735252078614
3.77588794397199 0.000319867594199996
3.77988994497249 0.000315067854490059
3.78389194597299 0.000310335166400483
3.78789394697349 0.000305668673195205
3.79189594797399 0.000301067527857504
3.79589794897449 0.000296530892997707
3.79989994997499 0.000292057940761512
3.80390195097549 0.000287647852738947
3.80790395197599 0.00028329981987394
3.81190595297649 0.000279013042374511
3.81590795397699 0.000274786729623591
3.81990995497749 0.000270620100090461
3.82391195597799 0.000266512381242796
3.82791395697849 0.000262462809459343
3.83191595797899 0.000258470629943209
3.83591795897949 0.000254535096635763
3.83991995997999 0.000250655472131154
3.84392196098049 0.000246831027591447
3.84792396198099 0.000243061042662359
3.85192596298149 0.00023934480538962
3.85592796398199 0.000235681612135933
3.85992996498249 0.000232070767498548
3.86393196598299 0.000228511584227436
3.86793396698349 0.000225003383144077
3.87193596798399 0.000221545493060848
3.87593796898449 0.000218137250701003
3.87993996998499 0.000214778000619271
3.88394197098549 0.000211467095123042
3.88794397198599 0.00020820389419415
3.89194597298649 0.000204987765411253
3.89594797398699 0.000201818083872808
3.89994997498749 0.000198694232120639
3.90395197598799 0.000195615600064085
3.90795397698849 0.000192581584904751
3.91195597798899 0.000189591591061833
3.91595797898949 0.000186645030098031
3.91995997998999 0.000183741320646047
3.92396198099049 0.000180879888335659
3.927963981991 0.000178060165721374
3.9319659829915 0.000175281592210657
3.935967983992 0.000172543613992736
3.9399699849925 0.000169845683967974
3.943971985993 0.000167187261677817
3.9479739869935 0.000164567813235301
3.951975987994 0.000161986811256128
3.9559779889945 0.000159443734790312
3.959979989995 0.00015693806925437
3.9639819909955 0.000154469306364083
3.967983991996 0.000152036944067809
3.9719859929965 0.000149640486480351
3.975987993997 0.000147279443817372
3.9799899949975 0.000144953332330358
3.983991995998 0.000142661674242141
3.9879939969985 0.000140403997682942
3.991995997999 0.000138179836626972
3.9959979989995 0.000135988730829575
4 0.000133830225764885
};
\addplot [line width=0.16pt, black]
table {%
1 0
1 0.241970724519143
};
\draw (axis cs:1,0.201642270432619) node[
  scale=0.5,
  anchor=east,
  text=black,
  rotate=0.0
]{0.8413  };
\end{axis}

\end{tikzpicture}
"""

In [ ]:
with open("ztable-input.tex", 'w') as f:
    f.write(s)
    
with open("ztable.tex", 'w') as f:
    doc = r"""\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage[margin=1in]{geometry}
\usepackage{booktabs}
\usepackage{colortbl}
\usepackage{graphicx}
\usepackage{xcolor}
\usepackage{tikz}
\usepackage{pgfplots}
\begin{document}
\begin{center}
\section*{$z$-table}
%\input{ztable-input.tex}
""" + s + r"""
\bigskip
%\includegraphics[width=.6\textwidth]{ztable_plot.pdf}
""" + tikz_plot + r"""
\end{center}
\end{document}
"""
    f.write(doc)

os.system("pdflatex ztable.tex")
os.system("open ztable.tex")
os.system(f"mv ztable.tex {wd}/ztable.tex")
os.system(f"mv ztable.pdf {wd}/ztable.pdf")
os.system(f"mv ztable-input.tex {wd}/tex/ztable-input.tex")

files = !ls
for f in files:
    if f.startswith("ztable."):
        os.system(f"rm {f}")
        #print(f)